In [1]:
# 导入所需的所有包
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import normaltest, skew, kurtosis
from scipy.stats import skew, kurtosis, norm
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
import os
import platform

# 根据操作系统设置合适的中文字体
system = platform.system()
if system == 'Darwin':  # macOS
    plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'Heiti TC', 'PingFang HK', 'Apple Color Emoji']
elif system == 'Windows':
    plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei', 'SimSun']
else:  # Linux或其他
    plt.rcParams['font.sans-serif'] = ['DejaVu Sans', 'WenQuanYi Micro Hei', 'WenQuanYi Zen Hei']

# 正常显示负号
plt.rcParams['axes.unicode_minus'] = False

# 检查字体是否设置成功
print(f"当前操作系统: {system}")
print(f"当前字体设置: {plt.rcParams['font.sans-serif']}")

当前操作系统: Windows
当前字体设置: ['SimHei', 'Microsoft YaHei', 'SimSun']


## 数据加载和初步检查

In [2]:
import os

# Asin_List_file = "旅行包-机会款式.xlsx" 

# # 文件和目录的设置
# input_file = Asin_List_file  # 这实际上是Excel文件

# 从Excel文件名中提取基础文件夹名称
folder_name = '生成结果/social_media/'
#folder_name = os.path.splitext(input_file)[0]
print(f"提取的文件夹名: {folder_name}")

# 确保文件夹存在
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"已创建文件夹: {folder_name}")

提取的文件夹名: 生成结果/social_media/


In [3]:
# 加载数据
data_file = os.path.join(folder_name, "数据变量语义匹配二元赋值结果.xlsx")
data = pd.read_excel(data_file)
print(f"数据加载完成，共 {len(data)} 行和 {len(data.columns)} 列")

# 修改IV文件路径
iv_file = os.path.join(folder_name, 'IV.txt')
with open(iv_file, 'r', encoding='utf-8') as f:
    iv_list = f.read().splitlines()
    iv_list = [iv.strip() for iv in iv_list if iv.strip()]  # 移除空行
print(f"从IV.txt加载了 {len(iv_list)} 个自变量")

# 定义因变量和控制变量
dv_col = 'interaction_view_cnt'
control_vars = ['author_followers_cnt', 'author_friends_cnt']
print(f"将使用的控制变量: {', '.join(control_vars)}")

# 检查DV列是否存在
if dv_col not in data.columns:
    print(f"错误: 数据中不存在列 '{dv_col}'")
    print("数据包含的列:", data.columns.tolist()[:10], "...")
    raise ValueError(f"找不到DV列: {dv_col}")
else:
    print(f"已确认因变量'{dv_col}'存在于数据中")

# 检查控制变量是否存在
missing_controls = [var for var in control_vars if var not in data.columns]
if missing_controls:
    print(f"错误: 以下控制变量在数据中不存在: {missing_controls}")
    raise ValueError("缺少必要的控制变量")
else:
    print("已确认所有控制变量存在于数据中")

# 检查IV列是否都存在
missing_ivs = [iv for iv in iv_list if iv not in data.columns]
if missing_ivs:
    print(f"警告: 以下IV在数据中不存在: {missing_ivs}")
    iv_list = [iv for iv in iv_list if iv in data.columns]
    print(f"将使用剩余的 {len(iv_list)} 个IV进行建模")

if not iv_list:
    raise ValueError("没有可用的IV变量进行建模")

# 修改结果文件路径
result_file = os.path.join(folder_name, "View-模型结果.txt")
with open(result_file, 'w', encoding='utf-8') as f:
    f.write("# 模型分析结果\n\n")
print(f"已创建结果文件: {result_file}")

# 显示数据前几行
print("数据预览:")
print(data.head())

数据加载完成，共 605 行和 67 列
从IV.txt加载了 18 个自变量
将使用的控制变量: author_followers_cnt, author_friends_cnt
已确认因变量'interaction_view_cnt'存在于数据中
已确认所有控制变量存在于数据中
已创建结果文件: 生成结果/social_media/View-模型结果.txt
数据预览:
                    id  title  \
0  1909480340126580992    NaN   
1  1909020699747886080    NaN   
2  1908203431765881088    NaN   
3  1908194727209385984    NaN   
4  1908053656731087104    NaN   

                                             content  url lang  \
0  Metal Shoe Rack\nUpto 24% off\nLink for you: h...  NaN   en   
1  @emob_ @DamnNearWhite @Tr3sMilagros @MyFirstKi...  NaN   en   
2  J.T. Foote Adjustable Shoe Trees - Plastic \n🛠...  NaN   en   
3  EXVITO Metal Shoe Rack for Home – Adjustable &...  NaN   en   
4  Closet Shoe Organizer for 24 Pairs with Adjust...  NaN   en   

                     publish_time publish_time_date publish_time_time  \
0  Tue Apr 08 05:36:04 +0000 2025        2025-04-08          05:36:04   
1  Sun Apr 06 23:09:37 +0000 2025        2025-04-06          23:09:37

## DV分布分析（简化模型标准）

In [4]:
# 分析DV的分布
def analyze_distribution(data, col):
    """
    分析变量分布并返回基本统计量和模型推荐
    """
    # 提取非空数据
    valid_data = data[col].dropna()
    
    # 基本统计量
    stats_dict = {
        "样本量": len(valid_data),
        "零值数量": (valid_data == 0).sum(),
        "零值比例": (valid_data == 0).mean() * 100,
        "均值": valid_data.mean(),
        "中位数": valid_data.median(),
        "标准差": valid_data.std(),
        "最小值": valid_data.min(),
        "最大值": valid_data.max(),
        "偏度": skew(valid_data),
        "峰度": kurtosis(valid_data)
    }

    # 模型选择逻辑 - 根据零值比例和分布偏度
    zero_inflated = stats_dict["零值比例"] > 20  # 如果零值超过20%，使用Tobit
    high_skew = abs(stats_dict["偏度"]) > 1.0   # 如果偏度较大，使用对数转换
    
    model_type = "ols"  # 默认模型类型
    transform_type = "none"  # 默认不转换
    reason = ""
    
    if zero_inflated:
        model_type = "tobit"
        reason += f"数据中零值比例为{stats_dict['零值比例']:.2f}%，超过20%，使用Tobit模型。"
        
        if high_skew:
            transform_type = "log"
            reason += f"数据偏度为{stats_dict['偏度']:.2f}，表现为偏态分布，对因变量进行对数转换。"
    else:
        model_type = "ols"
        reason += f"数据中零值比例为{stats_dict['零值比例']:.2f}%，不超过20%，使用OLS模型。"
        
        if high_skew:
            transform_type = "log"
            reason += f"数据偏度为{stats_dict['偏度']:.2f}，表现为偏态分布，对因变量进行对数转换。"
    
    return model_type, transform_type, reason, stats_dict

# 对DV进行分布分析
model_type, transform_type, reason, dv_stats = analyze_distribution(data, dv_col)

# 显示分析结果
print(f"\n## {dv_col} 分布分析结果:")
for key, value in dv_stats.items():
    if isinstance(value, float):
        print(f"- {key}: {value:.4f}")
    else:
        print(f"- {key}: {value}")

print(f"\n## 推荐的模型: {model_type.upper()}")
print(f"- 变量转换: {'对数转换' if transform_type == 'log' else '不转换'}")
print(f"- 原因: {reason}")

# 将结果保存到文件
with open(result_file, 'a', encoding='utf-8') as f:
    f.write(f"## {dv_col} 分布分析\n\n")
    for key, value in dv_stats.items():
        if isinstance(value, float):
            f.write(f"- {key}: {value:.4f}\n")
        else:
            f.write(f"- {key}: {value}\n")
    
    f.write(f"\n## 模型选择\n\n")
    f.write(f"- 推荐的模型类型: {model_type.upper()}\n")
    f.write(f"- 变量转换: {'对数转换' if transform_type == 'log' else '不转换'}\n")
    f.write(f"- 选择原因: {reason}\n\n")


## interaction_view_cnt 分布分析结果:
- 样本量: 605
- 零值数量: 21
- 零值比例: 3.4711
- 均值: 576.2678
- 中位数: 15.0000
- 标准差: 1830.7616
- 最小值: 0
- 最大值: 16742
- 偏度: 4.1096
- 峰度: 19.4342

## 推荐的模型: OLS
- 变量转换: 对数转换
- 原因: 数据中零值比例为3.47%，不超过20%，使用OLS模型。数据偏度为4.11，表现为偏态分布，对因变量进行对数转换。


## 数据预处理和模型准备

In [5]:
# 准备建模数据
all_predictors = iv_list + control_vars
model_data = data[[dv_col] + all_predictors].copy()
model_data = model_data.dropna()  # 移除有缺失值的行
print(f"预处理后的数据: {len(model_data)} 行 × {len(model_data.columns)} 列")

# 处理因变量转换
transformed_dv = dv_col
if transform_type == "log":
    # 检查零值和负值
    min_value = model_data[dv_col].min()
    if min_value <= 0:
        print(f"警告: {dv_col} 的最小值为 {min_value}，包含零值或负值")
        print("将使用log(1+x)转换")
        model_data[f'log_{dv_col}'] = np.log1p(model_data[dv_col])
    else:
        model_data[f'log_{dv_col}'] = np.log(model_data[dv_col])
    transformed_dv = f'log_{dv_col}'
    print(f"已创建对数转换变量: {transformed_dv}")
else:
    print(f"使用原始变量: {transformed_dv}")

# 构建公式并显示
formula = f"{transformed_dv} ~ " + " + ".join(all_predictors)
print(f"回归公式:")
print(formula)

# 检查多重共线性
if len(all_predictors) > 1:
    X = model_data[all_predictors]
    vif_data = pd.DataFrame()
    vif_data["变量"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    print("\n多重共线性检验 (VIF):")
    print("VIF > 10 表示可能存在严重的多重共线性问题")
    print(vif_data.sort_values("VIF", ascending=False))

    with open(result_file, 'a', encoding='utf-8') as f:
        f.write("## 多重共线性检验 (VIF)\n\n")
        f.write("VIF > 10 表示可能存在严重的多重共线性问题\n\n")
        f.write("| 变量 | VIF |\n")
        f.write("|------|------|\n")
        for index, row in vif_data.iterrows():
            f.write(f"| {row['变量']} | {row['VIF']:.4f} |\n")
        f.write("\n")


预处理后的数据: 605 行 × 21 列
警告: interaction_view_cnt 的最小值为 0，包含零值或负值
将使用log(1+x)转换
已创建对数转换变量: log_interaction_view_cnt
回归公式:
log_interaction_view_cnt ~ 安全性需求表达 + 空间效率表达 + 环保材质偏好 + 组装便捷性表达 + 设计美感表达 + 层高可调节性表达 + 防尘功能表达 + 价格表达 + 多功能性表达 + 便携性表达 + 承重能力表达 + 耐用性表达 + 颜色选择表达 + 季节适应性表达 + 品牌声誉表达 + 创新功能表达 + 儿童友好设计表达 + 组合灵活性表达 + author_followers_cnt + author_friends_cnt

多重共线性检验 (VIF):
VIF > 10 表示可能存在严重的多重共线性问题
                      变量       VIF
12                颜色选择表达  5.844991
4                 设计美感表达  5.432510
0                安全性需求表达  3.560953
8                 多功能性表达  3.394563
11                 耐用性表达  3.326069
3                组装便捷性表达  3.258193
2                 环保材质偏好  3.220954
10                承重能力表达  3.209544
13               季节适应性表达  3.190966
9                  便携性表达  3.183719
17               组合灵活性表达  3.163847
1                 空间效率表达  2.904204
15                创新功能表达  2.775960
14                品牌声誉表达  2.485333
6                 防尘功能表达  2.375646
16              儿童友好设计表达  2.106301
5        

## 模型拟合与结果输出

In [6]:
print(f"开始拟合{model_type.upper()}模型...")

if model_type == "tobit":
    # 尝试不同的导入方式
    try:
        # 尝试从truncated_model模块导入
        from statsmodels.discrete.truncated_model import Tobit
        
        # 准备数据
        y = model_data[transformed_dv].values
        X = sm.add_constant(model_data[all_predictors])
        
        print("使用statsmodels.discrete.truncated_model中的Tobit模型...")
        tobit_model = Tobit(y, X, left=0)
        results = tobit_model.fit(method='powell', disp=0, maxiter=10000)
        
    except (ImportError, AttributeError):
        # 如果上述导入失败，使用censored_model模块
        try:
            from statsmodels.regression.censored_model import Tobit
            
            # 准备数据
            y = model_data[transformed_dv].values
            X = sm.add_constant(model_data[all_predictors])
            
            print("使用statsmodels.regression.censored_model中的Tobit模型...")
            tobit_model = Tobit(y, X, left=0)
            results = tobit_model.fit(method='powell', disp=0, maxiter=10000)
            
        except (ImportError, AttributeError):
            # 如果上述方法都失败，使用改进的自定义实现
            print("使用改进的自定义Tobit模型实现...")
            from scipy.stats import norm
            from scipy import optimize
            from statsmodels.regression.linear_model import OLS
            from statsmodels.base.model import GenericLikelihoodModel
            
            class TobitModel(GenericLikelihoodModel):
                def __init__(self, endog, exog, left=0, **kwds):
                    self.left = left
                    super(TobitModel, self).__init__(endog, exog, **kwds)
                
                def loglikeobs(self, params):
                    beta = params[:-1]
                    sigma = np.abs(params[-1])  # 确保sigma为正
                    
                    q = self.endog
                    x = self.exog
                    
                    # 计算条件期望
                    mu = np.dot(x, beta)
                    
                    # 分别计算截尾和非截尾值的对数似然
                    censored_mask = (q <= self.left)
                    z = (self.left - mu) / sigma
                    
                    ll_censored = censored_mask * norm.logcdf(z)
                    
                    non_censored_mask = ~censored_mask
                    ll_non_censored = non_censored_mask * (
                        -np.log(sigma) + 
                        norm.logpdf((q - mu) / sigma)
                    )
                    
                    # 处理可能的数值问题
                    result = ll_censored + ll_non_censored
                    # 替换无效值
                    result = np.where(np.isnan(result) | np.isinf(result), -1e10, result)
                    
                    return result
                
                def nloglikeobs(self, params):
                    """负对数似然"""
                    return -self.loglikeobs(params)
                
                def fit(self, start_params=None, method='bfgs', maxiter=50000, **kwds):
                    """添加更多优化方法选项和更好的初始值策略"""
                    if start_params is None:
                        # 使用OLS估计获取更稳定的初始值
                        ols_model = OLS(
                            np.where(self.endog <= self.left, self.left, self.endog),
                            self.exog
                        )
                        ols_res = ols_model.fit()
                        # 使用残差的标准差作为sigma的初始值
                        start_params = np.append(ols_res.params, np.std(ols_res.resid))
                    
                    # 添加更多优化选项
                    if 'options' not in kwds:
                        kwds['options'] = {}
                    kwds['options']['maxiter'] = maxiter
                    
                    # 添加容错设置
                    try:
                        return super(TobitModel, self).fit(
                            start_params=start_params,
                            method=method, 
                            **kwds
                        )
                    except Exception as e:
                        print(f"首次优化失败: {str(e)}，尝试备用方法...")
                        try:
                            # 尝试Powell方法
                            return super(TobitModel, self).fit(
                                start_params=start_params,
                                method='powell',
                                options={'maxiter': 100000, 'ftol': 1e-8, 'xtol': 1e-8},
                                **kwds
                            )
                        except:
                            print("所有优化方法失败，尝试最简单的优化设置...")
                            return super(TobitModel, self).fit(
                                start_params=start_params,
                                method='nm',
                                options={'maxiter': 100000},
                                **kwds
                            )
            
            # 准备数据 - 修改数据准备部分以避免AttributeError
            y = model_data[transformed_dv].values
            X_df = sm.add_constant(model_data[all_predictors])  # 保持DataFrame格式
            X_array = X_df.values  # 数组版本用于拟合
            X_columns = X_df.columns.tolist()  # 保存列名供后续使用
            
            # 拟合Tobit模型，尝试多种方法
            tobit_model = TobitModel(y, X_array, left=0)
            try:
                # 首先尝试Nelder-Mead方法
                results = tobit_model.fit(method='nm', disp=0, maxiter=50000)
            except:
                try:
                    # 如果失败，尝试BFGS方法
                    results = tobit_model.fit(method='bfgs', disp=0, maxiter=50000)
                except:
                    # 最后尝试Powell方法
                    results = tobit_model.fit(method='powell', disp=0, maxiter=50000)
    
    # 打印结果
    print("\nTobit模型结果:")
    print(results.summary())
    
else:  # 使用OLS模型 - 保持不变
    # 创建并拟合模型
    model = smf.ols(formula=formula, data=model_data)
    results = model.fit()
    
    # 显示结果摘要
    print("\nOLS回归结果:")
    print(results.summary())

# 保存结果
with open(result_file, 'a', encoding='utf-8') as f:
    f.write(f"## {model_type.upper()}回归结果\n\n")
    f.write(f"被解释变量: {transformed_dv}\n\n")
    f.write("\n")
    f.write(str(results.summary()))
    f.write("\n\n\n")

# 模型结果解读
print("\n模型解读:")

if model_type == "ols":
    # OLS模型特有的解读 - 保持不变
    print(f"- 模型解释力 (R²): {results.rsquared:.4f}")
    print(f"- 调整后的R²: {results.rsquared_adj:.4f}")
    print(f"- 模型整体显著性: F({results.df_model:.0f},{results.df_resid:.0f})={results.fvalue:.4f}, p={results.f_pvalue:.6f}")
    
    if results.f_pvalue < 0.05:
        print("  结论: 模型整体上显著")
    else:
        print("  结论: 模型整体上不显著")
    
    # 获取显著的变量
    p_values = results.pvalues[1:]  # 跳过截距
    var_names = all_predictors
    
else:  # Tobit模型 - 修改此部分以处理可能的NaN问题
    print("- Tobit模型统计量:")
    print(f"  - 对数似然值: {results.llf:.4f}")
    print(f"  - AIC: {results.aic:.4f}")
    print(f"  - BIC: {results.bic:.4f}")
    
    # 安全获取Tobit模型的系数和p值
    if hasattr(results, 'pvalues') and not np.all(np.isnan(results.pvalues)):
        p_values = results.pvalues[1:-1]  # 跳过截距和sigma
    else:
        print("警告: p值计算失败，将使用系数作为显著性的近似指标")
        # 使用系数作为显著性的近似指标
        coefs = results.params[1:-1]
        se = np.ones_like(coefs) * 0.05  # 假设标准误为0.05
        p_values = 2 * (1 - norm.cdf(np.abs(coefs / se)))
    
    # 在自定义Tobit中X_columns已定义，在statsmodels版本中需要定义
    if 'X_columns' not in locals():
        X_columns = model_data[all_predictors].columns.tolist()
        X_columns = ['const'] + X_columns
    
    var_names = X_columns[1:-1] if model_type == "tobit" else X_columns[1:]  # 适应不同情况

# 显著变量分析（对两种模型通用）
significant_mask = p_values < 0.05
significant_vars = [var for var, sig in zip(var_names, significant_mask) if sig]
print(f"\n显著的变量 (p < 0.05): {len(significant_vars)}/{len(var_names)}")

iv_significant = [var for var in significant_vars if var in iv_list]
ctrl_significant = [var for var in significant_vars if var in control_vars]

# 显示显著的自变量
print(f"\n显著的自变量: {len(iv_significant)}/{len(iv_list)}")
for var in iv_significant:
    idx = var_names.index(var)
    coef = results.params[idx + 1]  # +1 to skip intercept
    p = p_values[idx]
    print(f"- {var}: 系数={coef:.4f}, p={p:.6f}")

# 显示显著的控制变量
print(f"\n显著的控制变量: {len(ctrl_significant)}/{len(control_vars)}")
for var in ctrl_significant:
    idx = var_names.index(var)
    coef = results.params[idx + 1]  # +1 to skip intercept
    p = p_values[idx]
    print(f"- {var}: 系数={coef:.4f}, p={p:.6f}")

# 保存结果
with open(result_file, 'a', encoding='utf-8') as f:
    f.write(f"## {model_type.upper()}回归结果\n\n")
    f.write(f"被解释变量: {transformed_dv}\n\n")
    f.write("\n")
    f.write(str(results.summary()))
    f.write("\n\n\n")

# 模型结果解读
print("\n模型解读:")

if model_type == "ols":
    # OLS模型特有的解读
    print(f"- 模型解释力 (R²): {results.rsquared:.4f}")
    print(f"- 调整后的R²: {results.rsquared_adj:.4f}")
    print(f"- 模型整体显著性: F({results.df_model:.0f},{results.df_resid:.0f})={results.fvalue:.4f}, p={results.f_pvalue:.6f}")
    
    if results.f_pvalue < 0.05:
        print("  结论: 模型整体上显著")
    else:
        print("  结论: 模型整体上不显著")
    
    # 获取显著的变量
    p_values = results.pvalues[1:]  # 跳过截距
    var_names = all_predictors
    
else:  # Tobit模型
    print("- Tobit模型统计量:")
    print(f"  - 对数似然值: {results.llf:.4f}")
    print(f"  - AIC: {results.aic:.4f}")
    print(f"  - BIC: {results.bic:.4f}")
    
    # Tobit模型中获取系数和p值
    p_values = results.pvalues[1:-1]  # 跳过截距和sigma
    var_names = list(X.columns[1:])  # 跳过常数项

# 显著变量分析（对两种模型通用）
significant_mask = p_values < 0.05
significant_vars = [var for var, sig in zip(var_names, significant_mask) if sig]
print(f"\n显著的变量 (p < 0.05): {len(significant_vars)}/{len(var_names)}")

iv_significant = [var for var in significant_vars if var in iv_list]
ctrl_significant = [var for var in significant_vars if var in control_vars]

# 显示显著的自变量
print(f"\n显著的自变量: {len(iv_significant)}/{len(iv_list)}")
for var in iv_significant:
    idx = var_names.index(var)
    coef = results.params[idx + 1]  # +1 to skip intercept
    p = p_values[idx]
    print(f"- {var}: 系数={coef:.4f}, p={p:.6f}")

# 显示显著的控制变量
print(f"\n显著的控制变量: {len(ctrl_significant)}/{len(control_vars)}")
for var in ctrl_significant:
    idx = var_names.index(var)
    coef = results.params[idx + 1]  # +1 to skip intercept
    p = p_values[idx]
    print(f"- {var}: 系数={coef:.4f}, p={p:.6f}")

# 保存解读结果
with open(result_file, 'a', encoding='utf-8') as f:
    f.write("## 模型结果解读\n\n")
    
    if model_type == "ols":
        f.write(f"- 模型解释力 (R²): {results.rsquared:.4f}\n")
        f.write(f"- 调整后的R²: {results.rsquared_adj:.4f}\n")
        f.write(f"- 模型整体显著性: F({results.df_model:.0f},{results.df_resid:.0f})={results.fvalue:.4f}, p={results.f_pvalue:.6f}\n")
        
        if results.f_pvalue < 0.05:
            f.write("  结论: 模型整体上显著\n\n")
        else:
            f.write("  结论: 模型整体上不显著\n\n")
    else:
        f.write("- Tobit模型统计量:\n")
        f.write(f"  - 对数似然值: {results.llf:.4f}\n")
        f.write(f"  - AIC: {results.aic:.4f}\n")
        f.write(f"  - BIC: {results.bic:.4f}\n\n")
    
    f.write(f"显著的变量 (p < 0.05): {len(significant_vars)}/{len(var_names)}\n\n")
    
    f.write(f"显著的自变量: {len(iv_significant)}/{len(iv_list)}\n")
    for var in iv_significant:
        idx = var_names.index(var)
        coef = results.params[idx + 1]
        p = p_values[idx]
        f.write(f"- {var}: 系数={coef:.4f}, p={p:.6f}\n")
    
    f.write(f"\n显著的控制变量: {len(ctrl_significant)}/{len(control_vars)}\n")
    for var in ctrl_significant:
        idx = var_names.index(var)
        coef = results.params[idx + 1]
        p = p_values[idx]
        f.write(f"- {var}: 系数={coef:.4f}, p={p:.6f}\n")

print(f"\n所有分析结果已保存到 {result_file}")


开始拟合OLS模型...

OLS回归结果:
                               OLS Regression Results                               
Dep. Variable:     log_interaction_view_cnt   R-squared:                       0.361
Model:                                  OLS   Adj. R-squared:                  0.339
Method:                       Least Squares   F-statistic:                     16.46
Date:                      Tue, 15 Apr 2025   Prob (F-statistic):           1.08e-44
Time:                              18:46:10   Log-Likelihood:                -1183.8
No. Observations:                       605   AIC:                             2410.
Df Residuals:                           584   BIC:                             2502.
Df Model:                                20                                         
Covariance Type:                  nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------

C:\Users\admin\AppData\Local\Temp\ipykernel_27916\1668281642.py:212: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coef = results.params[idx + 1]  # +1 to skip intercept
C:\Users\admin\AppData\Local\Temp\ipykernel_27916\1668281642.py:213: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p = p_values[idx]
C:\Users\admin\AppData\Local\Temp\ipykernel_27916\1668281642.py:220: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coef = results.par

## 系数归一化处理

In [7]:
# 归一化显著系数并输出结果
def normalize_significant_coefficients(results, iv_list, control_vars=None, model_type="ols", alpha=0.05):
    """
    归一化显著变量的系数，删除NaN值，按绝对值大小排序并输出结果
    
    参数:
    - results: 回归结果对象
    - iv_list: 自变量列表
    - control_vars: 控制变量列表，默认为None
    - model_type: 模型类型，"ols"或"tobit"
    - alpha: 显著性水平，默认0.05
    """
    if control_vars is None:
        control_vars = []
    
    all_vars = iv_list + control_vars
    
    # 获取系数和p值 - 需要考虑不同模型类型
    if model_type.lower() == "ols":
        # OLS模型 - 跳过截距
        coefs = results.params[1:len(all_vars)+1]
        p_values = results.pvalues[1:len(all_vars)+1]
    else:
        # Tobit模型 - 跳过截距和sigma
        coefs = results.params[1:len(all_vars)+1]
        p_values = results.pvalues[1:len(all_vars)+1]
    
    # 创建DataFrame
    coef_df = pd.DataFrame({
        '变量': all_vars,
        '系数': coefs,
        'p值': p_values,
        '变量类型': ['自变量' if var in iv_list else '控制变量' for var in all_vars]
    })
    
    # 过滤掉NaN系数和非显著的变量
    valid_coef_df = coef_df.dropna(subset=['系数'])
    sig_coef_df = valid_coef_df[valid_coef_df['p值'] < alpha].copy()
    
    # 如果没有显著变量，则报告所有有效变量
    if len(sig_coef_df) == 0:
        print("注意: 没有显著的变量 (p < 0.05)，将使用所有非NaN系数变量")
        sig_coef_df = valid_coef_df.copy()
    
    # 如果没有有效系数，退出
    if len(sig_coef_df) == 0:
        print("警告: 没有有效的系数，无法进行归一化")
        return None
    
    # 计算系数的绝对值
    sig_coef_df['系数绝对值'] = sig_coef_df['系数'].abs()
    
    # 归一化系数
    total_abs = sig_coef_df['系数绝对值'].sum()
    sig_coef_df['归一化系数'] = sig_coef_df['系数'] / total_abs
    sig_coef_df['归一化系数绝对值'] = sig_coef_df['系数绝对值'] / total_abs
    sig_coef_df['归一化系数百分比'] = sig_coef_df['归一化系数绝对值'] * 100
    
    # 按系数绝对值排序
    sig_coef_df = sig_coef_df.sort_values('系数绝对值', ascending=False)
    
    # 显示结果
    print("\n归一化后的系数 (按绝对值排序):")
    print("-----------------------------------")
    significant_str = "显著 (p<0.05)" if len(sig_coef_df[sig_coef_df['p值'] < alpha]) > 0 else "无显著变量"
    print(f"总计 {len(sig_coef_df)} 个有效变量, {significant_str}")
    print("-----------------------------------")
    
    # 先显示自变量，再显示控制变量
    for var_type in ['自变量', '控制变量']:
        type_df = sig_coef_df[sig_coef_df['变量类型'] == var_type]
        if len(type_df) > 0:
            print(f"\n{var_type}:")
            for _, row in type_df.iterrows():
                sig_mark = "*" if row['p值'] < alpha else ""
                print(f"{row['变量']}: {row['系数']:.4f} (归一化: {row['归一化系数']:.4f}, {row['归一化系数百分比']:.2f}%){sig_mark}")
    
    print("-----------------------------------")
    print("* 标记表示在0.05水平上显著")
    
    # 保存到文件
    output_file = os.path.join(folder_name, "View-归一化的模型结果.txt")
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(f"# {model_type.upper()} 模型归一化后的系数 (按绝对值排序)\n\n")
        f.write("-----------------------------------\n")
        f.write(f"总计 {len(sig_coef_df)} 个有效变量, {significant_str}\n")
        f.write("-----------------------------------\n\n")
        
        # 添加总表
        f.write("| 变量 | 变量类型 | 系数 | p值 | 归一化系数 | 归一化百分比 | 显著性 |\n")
        f.write("|------|---------|------|------|------------|------------|--------|\n")
        
        for _, row in sig_coef_df.iterrows():
            sig = "是" if row['p值'] < alpha else "否"
            f.write(f"| {row['变量']} | {row['变量类型']} | {row['系数']:.4f} | {row['p值']:.4f} | {row['归一化系数']:.4f} | {row['归一化系数百分比']:.2f}% | {sig} |\n")
        
        f.write("\n\n## 详细分析\n\n")
        
        # 先显示自变量，再显示控制变量
        for var_type in ['自变量', '控制变量']:
            type_df = sig_coef_df[sig_coef_df['变量类型'] == var_type]
            if len(type_df) > 0:
                f.write(f"### {var_type}\n\n")
                
                for _, row in type_df.iterrows():
                    sig_symbol = "*" if row['p值'] < alpha else ""
                    direction = "正向" if row['系数'] > 0 else "负向"
                    f.write(f"#### {row['变量']}{sig_symbol}\n\n")
                    f.write(f"- 系数: {row['系数']:.4f}\n")
                    f.write(f"- p值: {row['p值']:.4f}\n")
                    f.write(f"- 影响方向: {direction}\n")
                    f.write(f"- 归一化系数: {row['归一化系数']:.4f}\n")
                    f.write(f"- 归一化百分比: {row['归一化系数百分比']:.2f}%\n")
                    f.write(f"- 是否显著: {'是' if row['p值'] < alpha else '否'}\n\n")
        
        f.write("-----------------------------------\n")
        f.write("* 标记表示在0.05水平上显著\n")
    
    print(f"\n结果已保存到 {output_file}")
    
    return sig_coef_df
# 调用归一化函数
print("\n开始归一化系数分析...")
normalized_coefs = normalize_significant_coefficients(
    results=results,
    iv_list=iv_list,
    control_vars=control_vars,
    model_type=model_type,
    alpha=0.05
)



开始归一化系数分析...

归一化后的系数 (按绝对值排序):
-----------------------------------
总计 2 个有效变量, 显著 (p<0.05)
-----------------------------------

自变量:
层高可调节性表达: 0.4666 (归一化: 1.0000, 100.00%)*

控制变量:
author_followers_cnt: 0.0000 (归一化: 0.0000, 0.00%)*
-----------------------------------
* 标记表示在0.05水平上显著

结果已保存到 生成结果/social_media/View-归一化的模型结果.txt


## 其他变量分析

### 点赞分析

In [8]:
# 加载数据
data_file = os.path.join(folder_name, "数据变量语义匹配二元赋值结果.xlsx")
data = pd.read_excel(data_file)
print(f"数据加载完成，共 {len(data)} 行和 {len(data.columns)} 列")

# 修改IV文件路径
iv_file = os.path.join(folder_name, 'IV.txt')
with open(iv_file, 'r', encoding='utf-8') as f:
    iv_list = f.read().splitlines()
    iv_list = [iv.strip() for iv in iv_list if iv.strip()]  # 移除空行
print(f"从IV.txt加载了 {len(iv_list)} 个自变量")

# 定义因变量和控制变量
dv_col = 'interaction_like_cnt'
control_vars = ['author_followers_cnt', 'author_friends_cnt']
print(f"将使用的控制变量: {', '.join(control_vars)}")

# 检查DV列是否存在
if dv_col not in data.columns:
    print(f"错误: 数据中不存在列 '{dv_col}'")
    print("数据包含的列:", data.columns.tolist()[:10], "...")
    raise ValueError(f"找不到DV列: {dv_col}")
else:
    print(f"已确认因变量'{dv_col}'存在于数据中")

# 检查控制变量是否存在
missing_controls = [var for var in control_vars if var not in data.columns]
if missing_controls:
    print(f"错误: 以下控制变量在数据中不存在: {missing_controls}")
    raise ValueError("缺少必要的控制变量")
else:
    print("已确认所有控制变量存在于数据中")

# 检查IV列是否都存在
missing_ivs = [iv for iv in iv_list if iv not in data.columns]
if missing_ivs:
    print(f"警告: 以下IV在数据中不存在: {missing_ivs}")
    iv_list = [iv for iv in iv_list if iv in data.columns]
    print(f"将使用剩余的 {len(iv_list)} 个IV进行建模")

if not iv_list:
    raise ValueError("没有可用的IV变量进行建模")

# 修改结果文件路径
result_file = os.path.join(folder_name, "like-模型结果.txt")
with open(result_file, 'w', encoding='utf-8') as f:
    f.write("# 模型分析结果\n\n")
print(f"已创建结果文件: {result_file}")

# 显示数据前几行
print("数据预览:")
print(data.head())

# 分析DV的分布
def analyze_distribution(data, col):
    """
    分析变量分布并返回基本统计量和模型推荐
    """
    # 提取非空数据
    valid_data = data[col].dropna()
    
    # 基本统计量
    stats_dict = {
        "样本量": len(valid_data),
        "零值数量": (valid_data == 0).sum(),
        "零值比例": (valid_data == 0).mean() * 100,
        "均值": valid_data.mean(),
        "中位数": valid_data.median(),
        "标准差": valid_data.std(),
        "最小值": valid_data.min(),
        "最大值": valid_data.max(),
        "偏度": skew(valid_data),
        "峰度": kurtosis(valid_data)
    }

    # 模型选择逻辑 - 根据零值比例和分布偏度
    zero_inflated = stats_dict["零值比例"] > 20  # 如果零值超过20%，使用Tobit
    high_skew = abs(stats_dict["偏度"]) > 1.0   # 如果偏度较大，使用对数转换
    
    model_type = "ols"  # 默认模型类型
    transform_type = "none"  # 默认不转换
    reason = ""
    
    if zero_inflated:
        model_type = "tobit"
        reason += f"数据中零值比例为{stats_dict['零值比例']:.2f}%，超过20%，使用Tobit模型。"
        
        if high_skew:
            transform_type = "log"
            reason += f"数据偏度为{stats_dict['偏度']:.2f}，表现为偏态分布，对因变量进行对数转换。"
    else:
        model_type = "ols"
        reason += f"数据中零值比例为{stats_dict['零值比例']:.2f}%，不超过20%，使用OLS模型。"
        
        if high_skew:
            transform_type = "log"
            reason += f"数据偏度为{stats_dict['偏度']:.2f}，表现为偏态分布，对因变量进行对数转换。"
    
    return model_type, transform_type, reason, stats_dict

# 对DV进行分布分析
model_type, transform_type, reason, dv_stats = analyze_distribution(data, dv_col)

# 显示分析结果
print(f"\n## {dv_col} 分布分析结果:")
for key, value in dv_stats.items():
    if isinstance(value, float):
        print(f"- {key}: {value:.4f}")
    else:
        print(f"- {key}: {value}")

print(f"\n## 推荐的模型: {model_type.upper()}")
print(f"- 变量转换: {'对数转换' if transform_type == 'log' else '不转换'}")
print(f"- 原因: {reason}")

# 将结果保存到文件
with open(result_file, 'a', encoding='utf-8') as f:
    f.write(f"## {dv_col} 分布分析\n\n")
    for key, value in dv_stats.items():
        if isinstance(value, float):
            f.write(f"- {key}: {value:.4f}\n")
        else:
            f.write(f"- {key}: {value}\n")
    
    f.write(f"\n## 模型选择\n\n")
    f.write(f"- 推荐的模型类型: {model_type.upper()}\n")
    f.write(f"- 变量转换: {'对数转换' if transform_type == 'log' else '不转换'}\n")
    f.write(f"- 选择原因: {reason}\n\n")

# 准备建模数据
all_predictors = iv_list + control_vars
model_data = data[[dv_col] + all_predictors].copy()
model_data = model_data.dropna()  # 移除有缺失值的行
print(f"预处理后的数据: {len(model_data)} 行 × {len(model_data.columns)} 列")

# 处理因变量转换
transformed_dv = dv_col
if transform_type == "log":
    # 检查零值和负值
    min_value = model_data[dv_col].min()
    if min_value <= 0:
        print(f"警告: {dv_col} 的最小值为 {min_value}，包含零值或负值")
        print("将使用log(1+x)转换")
        model_data[f'log_{dv_col}'] = np.log1p(model_data[dv_col])
    else:
        model_data[f'log_{dv_col}'] = np.log(model_data[dv_col])
    transformed_dv = f'log_{dv_col}'
    print(f"已创建对数转换变量: {transformed_dv}")
else:
    print(f"使用原始变量: {transformed_dv}")

# 构建公式并显示
formula = f"{transformed_dv} ~ " + " + ".join(all_predictors)
print(f"回归公式:")
print(formula)

# 检查多重共线性
if len(all_predictors) > 1:
    X = model_data[all_predictors]
    vif_data = pd.DataFrame()
    vif_data["变量"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    print("\n多重共线性检验 (VIF):")
    print("VIF > 10 表示可能存在严重的多重共线性问题")
    print(vif_data.sort_values("VIF", ascending=False))

    with open(result_file, 'a', encoding='utf-8') as f:
        f.write("## 多重共线性检验 (VIF)\n\n")
        f.write("VIF > 10 表示可能存在严重的多重共线性问题\n\n")
        f.write("| 变量 | VIF |\n")
        f.write("|------|------|\n")
        for index, row in vif_data.iterrows():
            f.write(f"| {row['变量']} | {row['VIF']:.4f} |\n")
        f.write("\n")



数据加载完成，共 605 行和 67 列
从IV.txt加载了 18 个自变量
将使用的控制变量: author_followers_cnt, author_friends_cnt
已确认因变量'interaction_like_cnt'存在于数据中
已确认所有控制变量存在于数据中
已创建结果文件: 生成结果/social_media/like-模型结果.txt
数据预览:
                    id  title  \
0  1909480340126580992    NaN   
1  1909020699747886080    NaN   
2  1908203431765881088    NaN   
3  1908194727209385984    NaN   
4  1908053656731087104    NaN   

                                             content  url lang  \
0  Metal Shoe Rack\nUpto 24% off\nLink for you: h...  NaN   en   
1  @emob_ @DamnNearWhite @Tr3sMilagros @MyFirstKi...  NaN   en   
2  J.T. Foote Adjustable Shoe Trees - Plastic \n🛠...  NaN   en   
3  EXVITO Metal Shoe Rack for Home – Adjustable &...  NaN   en   
4  Closet Shoe Organizer for 24 Pairs with Adjust...  NaN   en   

                     publish_time publish_time_date publish_time_time  \
0  Tue Apr 08 05:36:04 +0000 2025        2025-04-08          05:36:04   
1  Sun Apr 06 23:09:37 +0000 2025        2025-04-06          23:09:37

In [9]:
print(f"开始拟合{model_type.upper()}模型...")

if model_type == "tobit":
    # 尝试不同的导入方式
    try:
        # 尝试从truncated_model模块导入
        from statsmodels.discrete.truncated_model import Tobit
        
        # 准备数据
        y = model_data[transformed_dv].values
        X = sm.add_constant(model_data[all_predictors])
        
        print("使用statsmodels.discrete.truncated_model中的Tobit模型...")
        tobit_model = Tobit(y, X, left=0)
        results = tobit_model.fit(method='powell', disp=0, maxiter=10000)
        
    except (ImportError, AttributeError):
        # 如果上述导入失败，使用censored_model模块
        try:
            from statsmodels.regression.censored_model import Tobit
            
            # 准备数据
            y = model_data[transformed_dv].values
            X = sm.add_constant(model_data[all_predictors])
            
            print("使用statsmodels.regression.censored_model中的Tobit模型...")
            tobit_model = Tobit(y, X, left=0)
            results = tobit_model.fit(method='powell', disp=0, maxiter=10000)
            
        except (ImportError, AttributeError):
            # 如果上述方法都失败，使用改进的自定义实现
            print("使用改进的自定义Tobit模型实现...")
            from scipy.stats import norm
            from scipy import optimize
            from statsmodels.regression.linear_model import OLS
            from statsmodels.base.model import GenericLikelihoodModel
            
            class TobitModel(GenericLikelihoodModel):
                def __init__(self, endog, exog, left=0, **kwds):
                    self.left = left
                    super(TobitModel, self).__init__(endog, exog, **kwds)
                
                def loglikeobs(self, params):
                    beta = params[:-1]
                    sigma = np.abs(params[-1])  # 确保sigma为正
                    
                    q = self.endog
                    x = self.exog
                    
                    # 计算条件期望
                    mu = np.dot(x, beta)
                    
                    # 分别计算截尾和非截尾值的对数似然
                    censored_mask = (q <= self.left)
                    z = (self.left - mu) / sigma
                    
                    ll_censored = censored_mask * norm.logcdf(z)
                    
                    non_censored_mask = ~censored_mask
                    ll_non_censored = non_censored_mask * (
                        -np.log(sigma) + 
                        norm.logpdf((q - mu) / sigma)
                    )
                    
                    # 处理可能的数值问题
                    result = ll_censored + ll_non_censored
                    # 替换无效值
                    result = np.where(np.isnan(result) | np.isinf(result), -1e10, result)
                    
                    return result
                
                def nloglikeobs(self, params):
                    """负对数似然"""
                    return -self.loglikeobs(params)
                
                def fit(self, start_params=None, method='bfgs', maxiter=50000, **kwds):
                    """添加更多优化方法选项和更好的初始值策略"""
                    if start_params is None:
                        # 使用OLS估计获取更稳定的初始值
                        ols_model = OLS(
                            np.where(self.endog <= self.left, self.left, self.endog),
                            self.exog
                        )
                        ols_res = ols_model.fit()
                        # 使用残差的标准差作为sigma的初始值
                        start_params = np.append(ols_res.params, np.std(ols_res.resid))
                    
                    # 添加更多优化选项
                    if 'options' not in kwds:
                        kwds['options'] = {}
                    kwds['options']['maxiter'] = maxiter
                    
                    # 添加容错设置
                    try:
                        return super(TobitModel, self).fit(
                            start_params=start_params,
                            method=method, 
                            **kwds
                        )
                    except Exception as e:
                        print(f"首次优化失败: {str(e)}，尝试备用方法...")
                        try:
                            # 尝试Powell方法
                            return super(TobitModel, self).fit(
                                start_params=start_params,
                                method='powell',
                                options={'maxiter': 100000, 'ftol': 1e-8, 'xtol': 1e-8},
                                **kwds
                            )
                        except:
                            print("所有优化方法失败，尝试最简单的优化设置...")
                            return super(TobitModel, self).fit(
                                start_params=start_params,
                                method='nm',
                                options={'maxiter': 100000},
                                **kwds
                            )
            
            # 准备数据 - 修改数据准备部分以避免AttributeError
            y = model_data[transformed_dv].values
            X_df = sm.add_constant(model_data[all_predictors])  # 保持DataFrame格式
            X_array = X_df.values  # 数组版本用于拟合
            X_columns = X_df.columns.tolist()  # 保存列名供后续使用
            
            # 拟合Tobit模型，尝试多种方法
            tobit_model = TobitModel(y, X_array, left=0)
            try:
                # 首先尝试Nelder-Mead方法
                results = tobit_model.fit(method='nm', disp=0, maxiter=50000)
            except:
                try:
                    # 如果失败，尝试BFGS方法
                    results = tobit_model.fit(method='bfgs', disp=0, maxiter=50000)
                except:
                    # 最后尝试Powell方法
                    results = tobit_model.fit(method='powell', disp=0, maxiter=50000)
    
    # 打印结果
    print("\nTobit模型结果:")
    print(results.summary())
    
else:  # 使用OLS模型 - 保持不变
    # 创建并拟合模型
    model = smf.ols(formula=formula, data=model_data)
    results = model.fit()
    
    # 显示结果摘要
    print("\nOLS回归结果:")
    print(results.summary())

# 保存结果
with open(result_file, 'a', encoding='utf-8') as f:
    f.write(f"## {model_type.upper()}回归结果\n\n")
    f.write(f"被解释变量: {transformed_dv}\n\n")
    f.write("\n")
    f.write(str(results.summary()))
    f.write("\n\n\n")

# 模型结果解读
print("\n模型解读:")

if model_type == "ols":
    # OLS模型特有的解读 - 保持不变
    print(f"- 模型解释力 (R²): {results.rsquared:.4f}")
    print(f"- 调整后的R²: {results.rsquared_adj:.4f}")
    print(f"- 模型整体显著性: F({results.df_model:.0f},{results.df_resid:.0f})={results.fvalue:.4f}, p={results.f_pvalue:.6f}")
    
    if results.f_pvalue < 0.05:
        print("  结论: 模型整体上显著")
    else:
        print("  结论: 模型整体上不显著")
    
    # 获取显著的变量
    p_values = results.pvalues[1:]  # 跳过截距
    var_names = all_predictors
    
else:  # Tobit模型 - 修改此部分以处理可能的NaN问题
    print("- Tobit模型统计量:")
    print(f"  - 对数似然值: {results.llf:.4f}")
    print(f"  - AIC: {results.aic:.4f}")
    print(f"  - BIC: {results.bic:.4f}")
    
    # 安全获取Tobit模型的系数和p值
    if hasattr(results, 'pvalues') and not np.all(np.isnan(results.pvalues)):
        p_values = results.pvalues[1:-1]  # 跳过截距和sigma
    else:
        print("警告: p值计算失败，将使用系数作为显著性的近似指标")
        # 使用系数作为显著性的近似指标
        coefs = results.params[1:-1]
        se = np.ones_like(coefs) * 0.05  # 假设标准误为0.05
        p_values = 2 * (1 - norm.cdf(np.abs(coefs / se)))
    
    # 在自定义Tobit中X_columns已定义，在statsmodels版本中需要定义
    if 'X_columns' not in locals():
        X_columns = model_data[all_predictors].columns.tolist()
        X_columns = ['const'] + X_columns
    
    var_names = X_columns[1:-1] if model_type == "tobit" else X_columns[1:]  # 适应不同情况

# 显著变量分析（对两种模型通用）
significant_mask = p_values < 0.05
significant_vars = [var for var, sig in zip(var_names, significant_mask) if sig]
print(f"\n显著的变量 (p < 0.05): {len(significant_vars)}/{len(var_names)}")

iv_significant = [var for var in significant_vars if var in iv_list]
ctrl_significant = [var for var in significant_vars if var in control_vars]

# 显示显著的自变量
print(f"\n显著的自变量: {len(iv_significant)}/{len(iv_list)}")
for var in iv_significant:
    idx = var_names.index(var)
    coef = results.params[idx + 1]  # +1 to skip intercept
    p = p_values[idx]
    print(f"- {var}: 系数={coef:.4f}, p={p:.6f}")

# 显示显著的控制变量
print(f"\n显著的控制变量: {len(ctrl_significant)}/{len(control_vars)}")
for var in ctrl_significant:
    idx = var_names.index(var)
    coef = results.params[idx + 1]  # +1 to skip intercept
    p = p_values[idx]
    print(f"- {var}: 系数={coef:.4f}, p={p:.6f}")

# 保存结果
with open(result_file, 'a', encoding='utf-8') as f:
    f.write(f"## {model_type.upper()}回归结果\n\n")
    f.write(f"被解释变量: {transformed_dv}\n\n")
    f.write("\n")
    f.write(str(results.summary()))
    f.write("\n\n\n")

# 模型结果解读
print("\n模型解读:")

if model_type == "ols":
    # OLS模型特有的解读
    print(f"- 模型解释力 (R²): {results.rsquared:.4f}")
    print(f"- 调整后的R²: {results.rsquared_adj:.4f}")
    print(f"- 模型整体显著性: F({results.df_model:.0f},{results.df_resid:.0f})={results.fvalue:.4f}, p={results.f_pvalue:.6f}")
    
    if results.f_pvalue < 0.05:
        print("  结论: 模型整体上显著")
    else:
        print("  结论: 模型整体上不显著")
    
    # 获取显著的变量
    p_values = results.pvalues[1:]  # 跳过截距
    var_names = all_predictors
    
else:  # Tobit模型
    print("- Tobit模型统计量:")
    print(f"  - 对数似然值: {results.llf:.4f}")
    print(f"  - AIC: {results.aic:.4f}")
    print(f"  - BIC: {results.bic:.4f}")
    
    # Tobit模型中获取系数和p值
    p_values = results.pvalues[1:-1]  # 跳过截距和sigma
    var_names = list(X.columns[1:])  # 跳过常数项

# 显著变量分析（对两种模型通用）
significant_mask = p_values < 0.05
significant_vars = [var for var, sig in zip(var_names, significant_mask) if sig]
print(f"\n显著的变量 (p < 0.05): {len(significant_vars)}/{len(var_names)}")

iv_significant = [var for var in significant_vars if var in iv_list]
ctrl_significant = [var for var in significant_vars if var in control_vars]

# 显示显著的自变量
print(f"\n显著的自变量: {len(iv_significant)}/{len(iv_list)}")
for var in iv_significant:
    idx = var_names.index(var)
    coef = results.params[idx + 1]  # +1 to skip intercept
    p = p_values[idx]
    print(f"- {var}: 系数={coef:.4f}, p={p:.6f}")

# 显示显著的控制变量
print(f"\n显著的控制变量: {len(ctrl_significant)}/{len(control_vars)}")
for var in ctrl_significant:
    idx = var_names.index(var)
    coef = results.params[idx + 1]  # +1 to skip intercept
    p = p_values[idx]
    print(f"- {var}: 系数={coef:.4f}, p={p:.6f}")

# 保存解读结果
with open(result_file, 'a', encoding='utf-8') as f:
    f.write("## 模型结果解读\n\n")
    
    if model_type == "ols":
        f.write(f"- 模型解释力 (R²): {results.rsquared:.4f}\n")
        f.write(f"- 调整后的R²: {results.rsquared_adj:.4f}\n")
        f.write(f"- 模型整体显著性: F({results.df_model:.0f},{results.df_resid:.0f})={results.fvalue:.4f}, p={results.f_pvalue:.6f}\n")
        
        if results.f_pvalue < 0.05:
            f.write("  结论: 模型整体上显著\n\n")
        else:
            f.write("  结论: 模型整体上不显著\n\n")
    else:
        f.write("- Tobit模型统计量:\n")
        f.write(f"  - 对数似然值: {results.llf:.4f}\n")
        f.write(f"  - AIC: {results.aic:.4f}\n")
        f.write(f"  - BIC: {results.bic:.4f}\n\n")
    
    f.write(f"显著的变量 (p < 0.05): {len(significant_vars)}/{len(var_names)}\n\n")
    
    f.write(f"显著的自变量: {len(iv_significant)}/{len(iv_list)}\n")
    for var in iv_significant:
        idx = var_names.index(var)
        coef = results.params[idx + 1]
        p = p_values[idx]
        f.write(f"- {var}: 系数={coef:.4f}, p={p:.6f}\n")
    
    f.write(f"\n显著的控制变量: {len(ctrl_significant)}/{len(control_vars)}\n")
    for var in ctrl_significant:
        idx = var_names.index(var)
        coef = results.params[idx + 1]
        p = p_values[idx]
        f.write(f"- {var}: 系数={coef:.4f}, p={p:.6f}\n")

print(f"\n所有分析结果已保存到 {result_file}")


开始拟合TOBIT模型...
使用改进的自定义Tobit模型实现...


C:\Users\admin\anaconda3\envs\Lynx\lib\site-packages\statsmodels\base\optimizer.py:19: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method nm is: xtol, ftol, maxfun. The list of unsupported keyword arguments passed include: options. After release 0.14, this will raise.
  warnings.warn(



Tobit模型结果:
                              TobitModel Results                              
Dep. Variable:                      y   Log-Likelihood:                -510.58
Model:                     TobitModel   AIC:                             1065.
Method:            Maximum Likelihood   BIC:                             1162.
Date:                Tue, 15 Apr 2025                                         
Time:                        18:46:10                                         
No. Observations:                 605                                         
Df Residuals:                     584                                         
Df Model:                          20                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4700      0.265     -1.774      0.076      -0.989       0.049
x1             0.0517      0.252      0.

C:\Users\admin\anaconda3\envs\Lynx\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\admin\anaconda3\envs\Lynx\lib\site-packages\statsmodels\base\model.py:2748: UserWarning: df_model + k_constant + k_extra differs from k_params
  warnings.warn("df_model + k_constant + k_extra "
C:\Users\admin\anaconda3\envs\Lynx\lib\site-packages\statsmodels\base\model.py:2752: UserWarning: df_resid differs from nobs - k_params
  warnings.warn("df_resid differs from nobs - k_params")


In [10]:
# 归一化显著系数并输出结果
def normalize_significant_coefficients(results, iv_list, control_vars=None, model_type="ols", alpha=0.05):
    """
    归一化显著变量的系数，删除NaN值，按绝对值大小排序并输出结果
    
    参数:
    - results: 回归结果对象
    - iv_list: 自变量列表
    - control_vars: 控制变量列表，默认为None
    - model_type: 模型类型，"ols"或"tobit"
    - alpha: 显著性水平，默认0.05
    """
    if control_vars is None:
        control_vars = []
    
    all_vars = iv_list + control_vars
    
    # 获取系数和p值 - 需要考虑不同模型类型
    if model_type.lower() == "ols":
        # OLS模型 - 跳过截距
        coefs = results.params[1:len(all_vars)+1]
        p_values = results.pvalues[1:len(all_vars)+1]
    else:
        # Tobit模型 - 跳过截距和sigma
        coefs = results.params[1:len(all_vars)+1]
        p_values = results.pvalues[1:len(all_vars)+1]
    
    # 创建DataFrame
    coef_df = pd.DataFrame({
        '变量': all_vars,
        '系数': coefs,
        'p值': p_values,
        '变量类型': ['自变量' if var in iv_list else '控制变量' for var in all_vars]
    })
    
    # 过滤掉NaN系数和非显著的变量
    valid_coef_df = coef_df.dropna(subset=['系数'])
    sig_coef_df = valid_coef_df[valid_coef_df['p值'] < alpha].copy()
    
    # 如果没有显著变量，则报告所有有效变量
    if len(sig_coef_df) == 0:
        print("注意: 没有显著的变量 (p < 0.05)，将使用所有非NaN系数变量")
        sig_coef_df = valid_coef_df.copy()
    
    # 如果没有有效系数，退出
    if len(sig_coef_df) == 0:
        print("警告: 没有有效的系数，无法进行归一化")
        return None
    
    # 计算系数的绝对值
    sig_coef_df['系数绝对值'] = sig_coef_df['系数'].abs()
    
    # 归一化系数
    total_abs = sig_coef_df['系数绝对值'].sum()
    sig_coef_df['归一化系数'] = sig_coef_df['系数'] / total_abs
    sig_coef_df['归一化系数绝对值'] = sig_coef_df['系数绝对值'] / total_abs
    sig_coef_df['归一化系数百分比'] = sig_coef_df['归一化系数绝对值'] * 100
    
    # 按系数绝对值排序
    sig_coef_df = sig_coef_df.sort_values('系数绝对值', ascending=False)
    
    # 显示结果
    print("\n归一化后的系数 (按绝对值排序):")
    print("-----------------------------------")
    significant_str = "显著 (p<0.05)" if len(sig_coef_df[sig_coef_df['p值'] < alpha]) > 0 else "无显著变量"
    print(f"总计 {len(sig_coef_df)} 个有效变量, {significant_str}")
    print("-----------------------------------")
    
    # 先显示自变量，再显示控制变量
    for var_type in ['自变量', '控制变量']:
        type_df = sig_coef_df[sig_coef_df['变量类型'] == var_type]
        if len(type_df) > 0:
            print(f"\n{var_type}:")
            for _, row in type_df.iterrows():
                sig_mark = "*" if row['p值'] < alpha else ""
                print(f"{row['变量']}: {row['系数']:.4f} (归一化: {row['归一化系数']:.4f}, {row['归一化系数百分比']:.2f}%){sig_mark}")
    
    print("-----------------------------------")
    print("* 标记表示在0.05水平上显著")
    
    # 保存到文件
    output_file = os.path.join(folder_name, "Like-归一化的模型结果.txt")
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(f"# {model_type.upper()} 模型归一化后的系数 (按绝对值排序)\n\n")
        f.write("-----------------------------------\n")
        f.write(f"总计 {len(sig_coef_df)} 个有效变量, {significant_str}\n")
        f.write("-----------------------------------\n\n")
        
        # 添加总表
        f.write("| 变量 | 变量类型 | 系数 | p值 | 归一化系数 | 归一化百分比 | 显著性 |\n")
        f.write("|------|---------|------|------|------------|------------|--------|\n")
        
        for _, row in sig_coef_df.iterrows():
            sig = "是" if row['p值'] < alpha else "否"
            f.write(f"| {row['变量']} | {row['变量类型']} | {row['系数']:.4f} | {row['p值']:.4f} | {row['归一化系数']:.4f} | {row['归一化系数百分比']:.2f}% | {sig} |\n")
        
        f.write("\n\n## 详细分析\n\n")
        
        # 先显示自变量，再显示控制变量
        for var_type in ['自变量', '控制变量']:
            type_df = sig_coef_df[sig_coef_df['变量类型'] == var_type]
            if len(type_df) > 0:
                f.write(f"### {var_type}\n\n")
                
                for _, row in type_df.iterrows():
                    sig_symbol = "*" if row['p值'] < alpha else ""
                    direction = "正向" if row['系数'] > 0 else "负向"
                    f.write(f"#### {row['变量']}{sig_symbol}\n\n")
                    f.write(f"- 系数: {row['系数']:.4f}\n")
                    f.write(f"- p值: {row['p值']:.4f}\n")
                    f.write(f"- 影响方向: {direction}\n")
                    f.write(f"- 归一化系数: {row['归一化系数']:.4f}\n")
                    f.write(f"- 归一化百分比: {row['归一化系数百分比']:.2f}%\n")
                    f.write(f"- 是否显著: {'是' if row['p值'] < alpha else '否'}\n\n")
        
        f.write("-----------------------------------\n")
        f.write("* 标记表示在0.05水平上显著\n")
    
    print(f"\n结果已保存到 {output_file}")
    
    return sig_coef_df
# 调用归一化函数
print("\n开始归一化系数分析...")
normalized_coefs = normalize_significant_coefficients(
    results=results,
    iv_list=iv_list,
    control_vars=control_vars,
    model_type=model_type,
    alpha=0.05
)



开始归一化系数分析...
注意: 没有显著的变量 (p < 0.05)，将使用所有非NaN系数变量

归一化后的系数 (按绝对值排序):
-----------------------------------
总计 20 个有效变量, 无显著变量
-----------------------------------

自变量:
多功能性表达: -0.4646 (归一化: -0.2304, 23.04%)
便携性表达: -0.2227 (归一化: -0.1105, 11.05%)
颜色选择表达: 0.2072 (归一化: 0.1028, 10.28%)
设计美感表达: -0.1978 (归一化: -0.0981, 9.81%)
空间效率表达: 0.1916 (归一化: 0.0950, 9.50%)
层高可调节性表达: 0.1807 (归一化: 0.0896, 8.96%)
环保材质偏好: -0.1599 (归一化: -0.0793, 7.93%)
价格表达: -0.0888 (归一化: -0.0440, 4.40%)
品牌声誉表达: -0.0801 (归一化: -0.0397, 3.97%)
防尘功能表达: -0.0544 (归一化: -0.0270, 2.70%)
安全性需求表达: 0.0517 (归一化: 0.0256, 2.56%)
组合灵活性表达: -0.0427 (归一化: -0.0212, 2.12%)
承重能力表达: 0.0323 (归一化: 0.0160, 1.60%)
组装便捷性表达: -0.0304 (归一化: -0.0151, 1.51%)
儿童友好设计表达: -0.0078 (归一化: -0.0039, 0.39%)
耐用性表达: -0.0022 (归一化: -0.0011, 0.11%)
创新功能表达: 0.0011 (归一化: 0.0006, 0.06%)
季节适应性表达: -0.0003 (归一化: -0.0001, 0.01%)

控制变量:
author_friends_cnt: 0.0000 (归一化: 0.0000, 0.00%)
author_followers_cnt: -0.0000 (归一化: -0.0000, 0.00%)
-----------------------------------
* 标记表示在0.05

### 评论数分析

In [11]:
# 加载数据
data_file = os.path.join(folder_name, "数据变量语义匹配二元赋值结果.xlsx")
data = pd.read_excel(data_file)
print(f"数据加载完成，共 {len(data)} 行和 {len(data.columns)} 列")

# 修改IV文件路径
iv_file = os.path.join(folder_name, 'IV.txt')
with open(iv_file, 'r', encoding='utf-8') as f:
    iv_list = f.read().splitlines()
    iv_list = [iv.strip() for iv in iv_list if iv.strip()]  # 移除空行
print(f"从IV.txt加载了 {len(iv_list)} 个自变量")

# 定义因变量和控制变量
dv_col = 'interaction_comment_cnt'
control_vars = ['author_followers_cnt', 'author_friends_cnt']
print(f"将使用的控制变量: {', '.join(control_vars)}")

# 检查DV列是否存在
if dv_col not in data.columns:
    print(f"错误: 数据中不存在列 '{dv_col}'")
    print("数据包含的列:", data.columns.tolist()[:10], "...")
    raise ValueError(f"找不到DV列: {dv_col}")
else:
    print(f"已确认因变量'{dv_col}'存在于数据中")

# 检查控制变量是否存在
missing_controls = [var for var in control_vars if var not in data.columns]
if missing_controls:
    print(f"错误: 以下控制变量在数据中不存在: {missing_controls}")
    raise ValueError("缺少必要的控制变量")
else:
    print("已确认所有控制变量存在于数据中")

# 检查IV列是否都存在
missing_ivs = [iv for iv in iv_list if iv not in data.columns]
if missing_ivs:
    print(f"警告: 以下IV在数据中不存在: {missing_ivs}")
    iv_list = [iv for iv in iv_list if iv in data.columns]
    print(f"将使用剩余的 {len(iv_list)} 个IV进行建模")

if not iv_list:
    raise ValueError("没有可用的IV变量进行建模")

# 修改结果文件路径
result_file = os.path.join(folder_name, "comment-模型结果.txt")
with open(result_file, 'w', encoding='utf-8') as f:
    f.write("# 模型分析结果\n\n")
print(f"已创建结果文件: {result_file}")

# 显示数据前几行
print("数据预览:")
print(data.head())

# 分析DV的分布
def analyze_distribution(data, col):
    """
    分析变量分布并返回基本统计量和模型推荐
    """
    # 提取非空数据
    valid_data = data[col].dropna()
    
    # 基本统计量
    stats_dict = {
        "样本量": len(valid_data),
        "零值数量": (valid_data == 0).sum(),
        "零值比例": (valid_data == 0).mean() * 100,
        "均值": valid_data.mean(),
        "中位数": valid_data.median(),
        "标准差": valid_data.std(),
        "最小值": valid_data.min(),
        "最大值": valid_data.max(),
        "偏度": skew(valid_data),
        "峰度": kurtosis(valid_data)
    }

    # 模型选择逻辑 - 根据零值比例和分布偏度
    zero_inflated = stats_dict["零值比例"] > 20  # 如果零值超过20%，使用Tobit
    high_skew = abs(stats_dict["偏度"]) > 1.0   # 如果偏度较大，使用对数转换
    
    model_type = "ols"  # 默认模型类型
    transform_type = "none"  # 默认不转换
    reason = ""
    
    if zero_inflated:
        model_type = "tobit"
        reason += f"数据中零值比例为{stats_dict['零值比例']:.2f}%，超过20%，使用Tobit模型。"
        
        if high_skew:
            transform_type = "log"
            reason += f"数据偏度为{stats_dict['偏度']:.2f}，表现为偏态分布，对因变量进行对数转换。"
    else:
        model_type = "ols"
        reason += f"数据中零值比例为{stats_dict['零值比例']:.2f}%，不超过20%，使用OLS模型。"
        
        if high_skew:
            transform_type = "log"
            reason += f"数据偏度为{stats_dict['偏度']:.2f}，表现为偏态分布，对因变量进行对数转换。"
    
    return model_type, transform_type, reason, stats_dict

# 对DV进行分布分析
model_type, transform_type, reason, dv_stats = analyze_distribution(data, dv_col)

# 显示分析结果
print(f"\n## {dv_col} 分布分析结果:")
for key, value in dv_stats.items():
    if isinstance(value, float):
        print(f"- {key}: {value:.4f}")
    else:
        print(f"- {key}: {value}")

print(f"\n## 推荐的模型: {model_type.upper()}")
print(f"- 变量转换: {'对数转换' if transform_type == 'log' else '不转换'}")
print(f"- 原因: {reason}")

# 将结果保存到文件
with open(result_file, 'a', encoding='utf-8') as f:
    f.write(f"## {dv_col} 分布分析\n\n")
    for key, value in dv_stats.items():
        if isinstance(value, float):
            f.write(f"- {key}: {value:.4f}\n")
        else:
            f.write(f"- {key}: {value}\n")
    
    f.write(f"\n## 模型选择\n\n")
    f.write(f"- 推荐的模型类型: {model_type.upper()}\n")
    f.write(f"- 变量转换: {'对数转换' if transform_type == 'log' else '不转换'}\n")
    f.write(f"- 选择原因: {reason}\n\n")

# 准备建模数据
all_predictors = iv_list + control_vars
model_data = data[[dv_col] + all_predictors].copy()
model_data = model_data.dropna()  # 移除有缺失值的行
print(f"预处理后的数据: {len(model_data)} 行 × {len(model_data.columns)} 列")

# 处理因变量转换
transformed_dv = dv_col
if transform_type == "log":
    # 检查零值和负值
    min_value = model_data[dv_col].min()
    if min_value <= 0:
        print(f"警告: {dv_col} 的最小值为 {min_value}，包含零值或负值")
        print("将使用log(1+x)转换")
        model_data[f'log_{dv_col}'] = np.log1p(model_data[dv_col])
    else:
        model_data[f'log_{dv_col}'] = np.log(model_data[dv_col])
    transformed_dv = f'log_{dv_col}'
    print(f"已创建对数转换变量: {transformed_dv}")
else:
    print(f"使用原始变量: {transformed_dv}")

# 构建公式并显示
formula = f"{transformed_dv} ~ " + " + ".join(all_predictors)
print(f"回归公式:")
print(formula)

# 检查多重共线性
if len(all_predictors) > 1:
    X = model_data[all_predictors]
    vif_data = pd.DataFrame()
    vif_data["变量"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    print("\n多重共线性检验 (VIF):")
    print("VIF > 10 表示可能存在严重的多重共线性问题")
    print(vif_data.sort_values("VIF", ascending=False))

    with open(result_file, 'a', encoding='utf-8') as f:
        f.write("## 多重共线性检验 (VIF)\n\n")
        f.write("VIF > 10 表示可能存在严重的多重共线性问题\n\n")
        f.write("| 变量 | VIF |\n")
        f.write("|------|------|\n")
        for index, row in vif_data.iterrows():
            f.write(f"| {row['变量']} | {row['VIF']:.4f} |\n")
        f.write("\n")



数据加载完成，共 605 行和 67 列
从IV.txt加载了 18 个自变量
将使用的控制变量: author_followers_cnt, author_friends_cnt
已确认因变量'interaction_comment_cnt'存在于数据中
已确认所有控制变量存在于数据中
已创建结果文件: 生成结果/social_media/comment-模型结果.txt
数据预览:
                    id  title  \
0  1909480340126580992    NaN   
1  1909020699747886080    NaN   
2  1908203431765881088    NaN   
3  1908194727209385984    NaN   
4  1908053656731087104    NaN   

                                             content  url lang  \
0  Metal Shoe Rack\nUpto 24% off\nLink for you: h...  NaN   en   
1  @emob_ @DamnNearWhite @Tr3sMilagros @MyFirstKi...  NaN   en   
2  J.T. Foote Adjustable Shoe Trees - Plastic \n🛠...  NaN   en   
3  EXVITO Metal Shoe Rack for Home – Adjustable &...  NaN   en   
4  Closet Shoe Organizer for 24 Pairs with Adjust...  NaN   en   

                     publish_time publish_time_date publish_time_time  \
0  Tue Apr 08 05:36:04 +0000 2025        2025-04-08          05:36:04   
1  Sun Apr 06 23:09:37 +0000 2025        2025-04-06          23

In [12]:
print(f"开始拟合{model_type.upper()}模型...")

if model_type == "tobit":
    # 尝试不同的导入方式
    try:
        # 尝试从truncated_model模块导入
        from statsmodels.discrete.truncated_model import Tobit
        
        # 准备数据
        y = model_data[transformed_dv].values
        X = sm.add_constant(model_data[all_predictors])
        
        print("使用statsmodels.discrete.truncated_model中的Tobit模型...")
        tobit_model = Tobit(y, X, left=0)
        results = tobit_model.fit(method='powell', disp=0, maxiter=10000)
        
    except (ImportError, AttributeError):
        # 如果上述导入失败，使用censored_model模块
        try:
            from statsmodels.regression.censored_model import Tobit
            
            # 准备数据
            y = model_data[transformed_dv].values
            X = sm.add_constant(model_data[all_predictors])
            
            print("使用statsmodels.regression.censored_model中的Tobit模型...")
            tobit_model = Tobit(y, X, left=0)
            results = tobit_model.fit(method='powell', disp=0, maxiter=10000)
            
        except (ImportError, AttributeError):
            # 如果上述方法都失败，使用改进的自定义实现
            print("使用改进的自定义Tobit模型实现...")
            from scipy.stats import norm
            from scipy import optimize
            from statsmodels.regression.linear_model import OLS
            from statsmodels.base.model import GenericLikelihoodModel
            
            class TobitModel(GenericLikelihoodModel):
                def __init__(self, endog, exog, left=0, **kwds):
                    self.left = left
                    super(TobitModel, self).__init__(endog, exog, **kwds)
                
                def loglikeobs(self, params):
                    beta = params[:-1]
                    sigma = np.abs(params[-1])  # 确保sigma为正
                    
                    q = self.endog
                    x = self.exog
                    
                    # 计算条件期望
                    mu = np.dot(x, beta)
                    
                    # 分别计算截尾和非截尾值的对数似然
                    censored_mask = (q <= self.left)
                    z = (self.left - mu) / sigma
                    
                    ll_censored = censored_mask * norm.logcdf(z)
                    
                    non_censored_mask = ~censored_mask
                    ll_non_censored = non_censored_mask * (
                        -np.log(sigma) + 
                        norm.logpdf((q - mu) / sigma)
                    )
                    
                    # 处理可能的数值问题
                    result = ll_censored + ll_non_censored
                    # 替换无效值
                    result = np.where(np.isnan(result) | np.isinf(result), -1e10, result)
                    
                    return result
                
                def nloglikeobs(self, params):
                    """负对数似然"""
                    return -self.loglikeobs(params)
                
                def fit(self, start_params=None, method='bfgs', maxiter=50000, **kwds):
                    """添加更多优化方法选项和更好的初始值策略"""
                    if start_params is None:
                        # 使用OLS估计获取更稳定的初始值
                        ols_model = OLS(
                            np.where(self.endog <= self.left, self.left, self.endog),
                            self.exog
                        )
                        ols_res = ols_model.fit()
                        # 使用残差的标准差作为sigma的初始值
                        start_params = np.append(ols_res.params, np.std(ols_res.resid))
                    
                    # 添加更多优化选项
                    if 'options' not in kwds:
                        kwds['options'] = {}
                    kwds['options']['maxiter'] = maxiter
                    
                    # 添加容错设置
                    try:
                        return super(TobitModel, self).fit(
                            start_params=start_params,
                            method=method, 
                            **kwds
                        )
                    except Exception as e:
                        print(f"首次优化失败: {str(e)}，尝试备用方法...")
                        try:
                            # 尝试Powell方法
                            return super(TobitModel, self).fit(
                                start_params=start_params,
                                method='powell',
                                options={'maxiter': 100000, 'ftol': 1e-8, 'xtol': 1e-8},
                                **kwds
                            )
                        except:
                            print("所有优化方法失败，尝试最简单的优化设置...")
                            return super(TobitModel, self).fit(
                                start_params=start_params,
                                method='nm',
                                options={'maxiter': 100000},
                                **kwds
                            )
            
            # 准备数据 - 修改数据准备部分以避免AttributeError
            y = model_data[transformed_dv].values
            X_df = sm.add_constant(model_data[all_predictors])  # 保持DataFrame格式
            X_array = X_df.values  # 数组版本用于拟合
            X_columns = X_df.columns.tolist()  # 保存列名供后续使用
            
            # 拟合Tobit模型，尝试多种方法
            tobit_model = TobitModel(y, X_array, left=0)
            try:
                # 首先尝试Nelder-Mead方法
                results = tobit_model.fit(method='nm', disp=0, maxiter=50000)
            except:
                try:
                    # 如果失败，尝试BFGS方法
                    results = tobit_model.fit(method='bfgs', disp=0, maxiter=50000)
                except:
                    # 最后尝试Powell方法
                    results = tobit_model.fit(method='powell', disp=0, maxiter=50000)
    
    # 打印结果
    print("\nTobit模型结果:")
    print(results.summary())
    
else:  # 使用OLS模型 - 保持不变
    # 创建并拟合模型
    model = smf.ols(formula=formula, data=model_data)
    results = model.fit()
    
    # 显示结果摘要
    print("\nOLS回归结果:")
    print(results.summary())

# 保存结果
with open(result_file, 'a', encoding='utf-8') as f:
    f.write(f"## {model_type.upper()}回归结果\n\n")
    f.write(f"被解释变量: {transformed_dv}\n\n")
    f.write("\n")
    f.write(str(results.summary()))
    f.write("\n\n\n")

# 模型结果解读
print("\n模型解读:")

if model_type == "ols":
    # OLS模型特有的解读 - 保持不变
    print(f"- 模型解释力 (R²): {results.rsquared:.4f}")
    print(f"- 调整后的R²: {results.rsquared_adj:.4f}")
    print(f"- 模型整体显著性: F({results.df_model:.0f},{results.df_resid:.0f})={results.fvalue:.4f}, p={results.f_pvalue:.6f}")
    
    if results.f_pvalue < 0.05:
        print("  结论: 模型整体上显著")
    else:
        print("  结论: 模型整体上不显著")
    
    # 获取显著的变量
    p_values = results.pvalues[1:]  # 跳过截距
    var_names = all_predictors
    
else:  # Tobit模型 - 修改此部分以处理可能的NaN问题
    print("- Tobit模型统计量:")
    print(f"  - 对数似然值: {results.llf:.4f}")
    print(f"  - AIC: {results.aic:.4f}")
    print(f"  - BIC: {results.bic:.4f}")
    
    # 安全获取Tobit模型的系数和p值
    if hasattr(results, 'pvalues') and not np.all(np.isnan(results.pvalues)):
        p_values = results.pvalues[1:-1]  # 跳过截距和sigma
    else:
        print("警告: p值计算失败，将使用系数作为显著性的近似指标")
        # 使用系数作为显著性的近似指标
        coefs = results.params[1:-1]
        se = np.ones_like(coefs) * 0.05  # 假设标准误为0.05
        p_values = 2 * (1 - norm.cdf(np.abs(coefs / se)))
    
    # 在自定义Tobit中X_columns已定义，在statsmodels版本中需要定义
    if 'X_columns' not in locals():
        X_columns = model_data[all_predictors].columns.tolist()
        X_columns = ['const'] + X_columns
    
    var_names = X_columns[1:-1] if model_type == "tobit" else X_columns[1:]  # 适应不同情况

# 显著变量分析（对两种模型通用）
significant_mask = p_values < 0.05
significant_vars = [var for var, sig in zip(var_names, significant_mask) if sig]
print(f"\n显著的变量 (p < 0.05): {len(significant_vars)}/{len(var_names)}")

iv_significant = [var for var in significant_vars if var in iv_list]
ctrl_significant = [var for var in significant_vars if var in control_vars]

# 显示显著的自变量
print(f"\n显著的自变量: {len(iv_significant)}/{len(iv_list)}")
for var in iv_significant:
    idx = var_names.index(var)
    coef = results.params[idx + 1]  # +1 to skip intercept
    p = p_values[idx]
    print(f"- {var}: 系数={coef:.4f}, p={p:.6f}")

# 显示显著的控制变量
print(f"\n显著的控制变量: {len(ctrl_significant)}/{len(control_vars)}")
for var in ctrl_significant:
    idx = var_names.index(var)
    coef = results.params[idx + 1]  # +1 to skip intercept
    p = p_values[idx]
    print(f"- {var}: 系数={coef:.4f}, p={p:.6f}")

# 保存结果
with open(result_file, 'a', encoding='utf-8') as f:
    f.write(f"## {model_type.upper()}回归结果\n\n")
    f.write(f"被解释变量: {transformed_dv}\n\n")
    f.write("\n")
    f.write(str(results.summary()))
    f.write("\n\n\n")

# 模型结果解读
print("\n模型解读:")

if model_type == "ols":
    # OLS模型特有的解读
    print(f"- 模型解释力 (R²): {results.rsquared:.4f}")
    print(f"- 调整后的R²: {results.rsquared_adj:.4f}")
    print(f"- 模型整体显著性: F({results.df_model:.0f},{results.df_resid:.0f})={results.fvalue:.4f}, p={results.f_pvalue:.6f}")
    
    if results.f_pvalue < 0.05:
        print("  结论: 模型整体上显著")
    else:
        print("  结论: 模型整体上不显著")
    
    # 获取显著的变量
    p_values = results.pvalues[1:]  # 跳过截距
    var_names = all_predictors
    
else:  # Tobit模型
    print("- Tobit模型统计量:")
    print(f"  - 对数似然值: {results.llf:.4f}")
    print(f"  - AIC: {results.aic:.4f}")
    print(f"  - BIC: {results.bic:.4f}")
    
    # Tobit模型中获取系数和p值
    p_values = results.pvalues[1:-1]  # 跳过截距和sigma
    var_names = list(X.columns[1:])  # 跳过常数项

# 显著变量分析（对两种模型通用）
significant_mask = p_values < 0.05
significant_vars = [var for var, sig in zip(var_names, significant_mask) if sig]
print(f"\n显著的变量 (p < 0.05): {len(significant_vars)}/{len(var_names)}")

iv_significant = [var for var in significant_vars if var in iv_list]
ctrl_significant = [var for var in significant_vars if var in control_vars]

# 显示显著的自变量
print(f"\n显著的自变量: {len(iv_significant)}/{len(iv_list)}")
for var in iv_significant:
    idx = var_names.index(var)
    coef = results.params[idx + 1]  # +1 to skip intercept
    p = p_values[idx]
    print(f"- {var}: 系数={coef:.4f}, p={p:.6f}")

# 显示显著的控制变量
print(f"\n显著的控制变量: {len(ctrl_significant)}/{len(control_vars)}")
for var in ctrl_significant:
    idx = var_names.index(var)
    coef = results.params[idx + 1]  # +1 to skip intercept
    p = p_values[idx]
    print(f"- {var}: 系数={coef:.4f}, p={p:.6f}")

# 保存解读结果
with open(result_file, 'a', encoding='utf-8') as f:
    f.write("## 模型结果解读\n\n")
    
    if model_type == "ols":
        f.write(f"- 模型解释力 (R²): {results.rsquared:.4f}\n")
        f.write(f"- 调整后的R²: {results.rsquared_adj:.4f}\n")
        f.write(f"- 模型整体显著性: F({results.df_model:.0f},{results.df_resid:.0f})={results.fvalue:.4f}, p={results.f_pvalue:.6f}\n")
        
        if results.f_pvalue < 0.05:
            f.write("  结论: 模型整体上显著\n\n")
        else:
            f.write("  结论: 模型整体上不显著\n\n")
    else:
        f.write("- Tobit模型统计量:\n")
        f.write(f"  - 对数似然值: {results.llf:.4f}\n")
        f.write(f"  - AIC: {results.aic:.4f}\n")
        f.write(f"  - BIC: {results.bic:.4f}\n\n")
    
    f.write(f"显著的变量 (p < 0.05): {len(significant_vars)}/{len(var_names)}\n\n")
    
    f.write(f"显著的自变量: {len(iv_significant)}/{len(iv_list)}\n")
    for var in iv_significant:
        idx = var_names.index(var)
        coef = results.params[idx + 1]
        p = p_values[idx]
        f.write(f"- {var}: 系数={coef:.4f}, p={p:.6f}\n")
    
    f.write(f"\n显著的控制变量: {len(ctrl_significant)}/{len(control_vars)}\n")
    for var in ctrl_significant:
        idx = var_names.index(var)
        coef = results.params[idx + 1]
        p = p_values[idx]
        f.write(f"- {var}: 系数={coef:.4f}, p={p:.6f}\n")

print(f"\n所有分析结果已保存到 {result_file}")


开始拟合TOBIT模型...
使用改进的自定义Tobit模型实现...


C:\Users\admin\anaconda3\envs\Lynx\lib\site-packages\statsmodels\base\optimizer.py:19: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method nm is: xtol, ftol, maxfun. The list of unsupported keyword arguments passed include: options. After release 0.14, this will raise.
  warnings.warn(



Tobit模型结果:
                              TobitModel Results                              
Dep. Variable:                      y   Log-Likelihood:                -257.69
Model:                     TobitModel   AIC:                             559.4
Method:            Maximum Likelihood   BIC:                             656.3
Date:                Tue, 15 Apr 2025                                         
Time:                        18:46:11                                         
No. Observations:                 605                                         
Df Residuals:                     584                                         
Df Model:                          20                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.3341        nan        nan        nan         nan         nan
x1             0.0123        nan        

C:\Users\admin\anaconda3\envs\Lynx\lib\site-packages\statsmodels\base\model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
C:\Users\admin\anaconda3\envs\Lynx\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\admin\anaconda3\envs\Lynx\lib\site-packages\statsmodels\base\model.py:2748: UserWarning: df_model + k_constant + k_extra differs from k_params
  warnings.warn("df_model + k_constant + k_extra "
C:\Users\admin\anaconda3\envs\Lynx\lib\site-packages\statsmodels\base\model.py:2752: UserWarning: df_resid differs from nobs - k_params
  warnings.warn("df_resid differs from nobs - k_params")


In [13]:
# 归一化显著系数并输出结果
def normalize_significant_coefficients(results, iv_list, control_vars=None, model_type="ols", alpha=0.05):
    """
    归一化显著变量的系数，删除NaN值，按绝对值大小排序并输出结果
    
    参数:
    - results: 回归结果对象
    - iv_list: 自变量列表
    - control_vars: 控制变量列表，默认为None
    - model_type: 模型类型，"ols"或"tobit"
    - alpha: 显著性水平，默认0.05
    """
    if control_vars is None:
        control_vars = []
    
    all_vars = iv_list + control_vars
    
    # 获取系数和p值 - 需要考虑不同模型类型
    if model_type.lower() == "ols":
        # OLS模型 - 跳过截距
        coefs = results.params[1:len(all_vars)+1]
        p_values = results.pvalues[1:len(all_vars)+1]
    else:
        # Tobit模型 - 跳过截距和sigma
        coefs = results.params[1:len(all_vars)+1]
        p_values = results.pvalues[1:len(all_vars)+1]
    
    # 创建DataFrame
    coef_df = pd.DataFrame({
        '变量': all_vars,
        '系数': coefs,
        'p值': p_values,
        '变量类型': ['自变量' if var in iv_list else '控制变量' for var in all_vars]
    })
    
    # 过滤掉NaN系数和非显著的变量
    valid_coef_df = coef_df.dropna(subset=['系数'])
    sig_coef_df = valid_coef_df[valid_coef_df['p值'] < alpha].copy()
    
    # 如果没有显著变量，则报告所有有效变量
    if len(sig_coef_df) == 0:
        print("注意: 没有显著的变量 (p < 0.05)，将使用所有非NaN系数变量")
        sig_coef_df = valid_coef_df.copy()
    
    # 如果没有有效系数，退出
    if len(sig_coef_df) == 0:
        print("警告: 没有有效的系数，无法进行归一化")
        return None
    
    # 计算系数的绝对值
    sig_coef_df['系数绝对值'] = sig_coef_df['系数'].abs()
    
    # 归一化系数
    total_abs = sig_coef_df['系数绝对值'].sum()
    sig_coef_df['归一化系数'] = sig_coef_df['系数'] / total_abs
    sig_coef_df['归一化系数绝对值'] = sig_coef_df['系数绝对值'] / total_abs
    sig_coef_df['归一化系数百分比'] = sig_coef_df['归一化系数绝对值'] * 100
    
    # 按系数绝对值排序
    sig_coef_df = sig_coef_df.sort_values('系数绝对值', ascending=False)
    
    # 显示结果
    print("\n归一化后的系数 (按绝对值排序):")
    print("-----------------------------------")
    significant_str = "显著 (p<0.05)" if len(sig_coef_df[sig_coef_df['p值'] < alpha]) > 0 else "无显著变量"
    print(f"总计 {len(sig_coef_df)} 个有效变量, {significant_str}")
    print("-----------------------------------")
    
    # 先显示自变量，再显示控制变量
    for var_type in ['自变量', '控制变量']:
        type_df = sig_coef_df[sig_coef_df['变量类型'] == var_type]
        if len(type_df) > 0:
            print(f"\n{var_type}:")
            for _, row in type_df.iterrows():
                sig_mark = "*" if row['p值'] < alpha else ""
                print(f"{row['变量']}: {row['系数']:.4f} (归一化: {row['归一化系数']:.4f}, {row['归一化系数百分比']:.2f}%){sig_mark}")
    
    print("-----------------------------------")
    print("* 标记表示在0.05水平上显著")
    
    # 保存到文件
    output_file = os.path.join(folder_name, "Comment-归一化的模型结果.txt")
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(f"# {model_type.upper()} 模型归一化后的系数 (按绝对值排序)\n\n")
        f.write("-----------------------------------\n")
        f.write(f"总计 {len(sig_coef_df)} 个有效变量, {significant_str}\n")
        f.write("-----------------------------------\n\n")
        
        # 添加总表
        f.write("| 变量 | 变量类型 | 系数 | p值 | 归一化系数 | 归一化百分比 | 显著性 |\n")
        f.write("|------|---------|------|------|------------|------------|--------|\n")
        
        for _, row in sig_coef_df.iterrows():
            sig = "是" if row['p值'] < alpha else "否"
            f.write(f"| {row['变量']} | {row['变量类型']} | {row['系数']:.4f} | {row['p值']:.4f} | {row['归一化系数']:.4f} | {row['归一化系数百分比']:.2f}% | {sig} |\n")
        
        f.write("\n\n## 详细分析\n\n")
        
        # 先显示自变量，再显示控制变量
        for var_type in ['自变量', '控制变量']:
            type_df = sig_coef_df[sig_coef_df['变量类型'] == var_type]
            if len(type_df) > 0:
                f.write(f"### {var_type}\n\n")
                
                for _, row in type_df.iterrows():
                    sig_symbol = "*" if row['p值'] < alpha else ""
                    direction = "正向" if row['系数'] > 0 else "负向"
                    f.write(f"#### {row['变量']}{sig_symbol}\n\n")
                    f.write(f"- 系数: {row['系数']:.4f}\n")
                    f.write(f"- p值: {row['p值']:.4f}\n")
                    f.write(f"- 影响方向: {direction}\n")
                    f.write(f"- 归一化系数: {row['归一化系数']:.4f}\n")
                    f.write(f"- 归一化百分比: {row['归一化系数百分比']:.2f}%\n")
                    f.write(f"- 是否显著: {'是' if row['p值'] < alpha else '否'}\n\n")
        
        f.write("-----------------------------------\n")
        f.write("* 标记表示在0.05水平上显著\n")
    
    print(f"\n结果已保存到 {output_file}")
    
    return sig_coef_df
# 调用归一化函数
print("\n开始归一化系数分析...")
normalized_coefs = normalize_significant_coefficients(
    results=results,
    iv_list=iv_list,
    control_vars=control_vars,
    model_type=model_type,
    alpha=0.05
)



开始归一化系数分析...
注意: 没有显著的变量 (p < 0.05)，将使用所有非NaN系数变量

归一化后的系数 (按绝对值排序):
-----------------------------------
总计 20 个有效变量, 无显著变量
-----------------------------------

自变量:
季节适应性表达: -0.2104 (归一化: -0.3447, 34.47%)
品牌声誉表达: 0.1089 (归一化: 0.1785, 17.85%)
价格表达: -0.0923 (归一化: -0.1512, 15.12%)
组合灵活性表达: 0.0464 (归一化: 0.0760, 7.60%)
设计美感表达: -0.0387 (归一化: -0.0634, 6.34%)
便携性表达: -0.0319 (归一化: -0.0523, 5.23%)
创新功能表达: -0.0283 (归一化: -0.0464, 4.64%)
安全性需求表达: 0.0123 (归一化: 0.0202, 2.02%)
组装便捷性表达: -0.0099 (归一化: -0.0161, 1.61%)
环保材质偏好: -0.0094 (归一化: -0.0154, 1.54%)
空间效率表达: 0.0072 (归一化: 0.0118, 1.18%)
耐用性表达: 0.0066 (归一化: 0.0109, 1.09%)
多功能性表达: 0.0029 (归一化: 0.0047, 0.47%)
层高可调节性表达: -0.0026 (归一化: -0.0043, 0.43%)
防尘功能表达: -0.0012 (归一化: -0.0020, 0.20%)
颜色选择表达: -0.0008 (归一化: -0.0014, 0.14%)
承重能力表达: -0.0002 (归一化: -0.0004, 0.04%)
儿童友好设计表达: -0.0002 (归一化: -0.0003, 0.03%)

控制变量:
author_friends_cnt: 0.0000 (归一化: 0.0000, 0.00%)
author_followers_cnt: 0.0000 (归一化: 0.0000, 0.00%)
-----------------------------------
* 标记表示在0.05水平

### 转帖分析

In [14]:
# 加载数据
data_file = os.path.join(folder_name, "数据变量语义匹配二元赋值结果.xlsx")
data = pd.read_excel(data_file)
print(f"数据加载完成，共 {len(data)} 行和 {len(data.columns)} 列")

# 修改IV文件路径
iv_file = os.path.join(folder_name, 'IV.txt')
with open(iv_file, 'r', encoding='utf-8') as f:
    iv_list = f.read().splitlines()
    iv_list = [iv.strip() for iv in iv_list if iv.strip()]  # 移除空行
print(f"从IV.txt加载了 {len(iv_list)} 个自变量")

# 定义因变量和控制变量
dv_col = 'interaction_repost_cnt'
control_vars = ['author_followers_cnt', 'author_friends_cnt']
print(f"将使用的控制变量: {', '.join(control_vars)}")

# 检查DV列是否存在
if dv_col not in data.columns:
    print(f"错误: 数据中不存在列 '{dv_col}'")
    print("数据包含的列:", data.columns.tolist()[:10], "...")
    raise ValueError(f"找不到DV列: {dv_col}")
else:
    print(f"已确认因变量'{dv_col}'存在于数据中")

# 检查控制变量是否存在
missing_controls = [var for var in control_vars if var not in data.columns]
if missing_controls:
    print(f"错误: 以下控制变量在数据中不存在: {missing_controls}")
    raise ValueError("缺少必要的控制变量")
else:
    print("已确认所有控制变量存在于数据中")

# 检查IV列是否都存在
missing_ivs = [iv for iv in iv_list if iv not in data.columns]
if missing_ivs:
    print(f"警告: 以下IV在数据中不存在: {missing_ivs}")
    iv_list = [iv for iv in iv_list if iv in data.columns]
    print(f"将使用剩余的 {len(iv_list)} 个IV进行建模")

if not iv_list:
    raise ValueError("没有可用的IV变量进行建模")

# 修改结果文件路径
result_file = os.path.join(folder_name, "repost-模型结果.txt")
with open(result_file, 'w', encoding='utf-8') as f:
    f.write("# 模型分析结果\n\n")
print(f"已创建结果文件: {result_file}")

# 显示数据前几行
print("数据预览:")
print(data.head())

# 分析DV的分布
def analyze_distribution(data, col):
    """
    分析变量分布并返回基本统计量和模型推荐
    """
    # 提取非空数据
    valid_data = data[col].dropna()
    
    # 基本统计量
    stats_dict = {
        "样本量": len(valid_data),
        "零值数量": (valid_data == 0).sum(),
        "零值比例": (valid_data == 0).mean() * 100,
        "均值": valid_data.mean(),
        "中位数": valid_data.median(),
        "标准差": valid_data.std(),
        "最小值": valid_data.min(),
        "最大值": valid_data.max(),
        "偏度": skew(valid_data),
        "峰度": kurtosis(valid_data)
    }

    # 模型选择逻辑 - 根据零值比例和分布偏度
    zero_inflated = stats_dict["零值比例"] > 20  # 如果零值超过20%，使用Tobit
    high_skew = abs(stats_dict["偏度"]) > 1.0   # 如果偏度较大，使用对数转换
    
    model_type = "ols"  # 默认模型类型
    transform_type = "none"  # 默认不转换
    reason = ""
    
    if zero_inflated:
        model_type = "tobit"
        reason += f"数据中零值比例为{stats_dict['零值比例']:.2f}%，超过20%，使用Tobit模型。"
        
        if high_skew:
            transform_type = "log"
            reason += f"数据偏度为{stats_dict['偏度']:.2f}，表现为偏态分布，对因变量进行对数转换。"
    else:
        model_type = "ols"
        reason += f"数据中零值比例为{stats_dict['零值比例']:.2f}%，不超过20%，使用OLS模型。"
        
        if high_skew:
            transform_type = "log"
            reason += f"数据偏度为{stats_dict['偏度']:.2f}，表现为偏态分布，对因变量进行对数转换。"
    
    return model_type, transform_type, reason, stats_dict

# 对DV进行分布分析
model_type, transform_type, reason, dv_stats = analyze_distribution(data, dv_col)

# 显示分析结果
print(f"\n## {dv_col} 分布分析结果:")
for key, value in dv_stats.items():
    if isinstance(value, float):
        print(f"- {key}: {value:.4f}")
    else:
        print(f"- {key}: {value}")

print(f"\n## 推荐的模型: {model_type.upper()}")
print(f"- 变量转换: {'对数转换' if transform_type == 'log' else '不转换'}")
print(f"- 原因: {reason}")

# 将结果保存到文件
with open(result_file, 'a', encoding='utf-8') as f:
    f.write(f"## {dv_col} 分布分析\n\n")
    for key, value in dv_stats.items():
        if isinstance(value, float):
            f.write(f"- {key}: {value:.4f}\n")
        else:
            f.write(f"- {key}: {value}\n")
    
    f.write(f"\n## 模型选择\n\n")
    f.write(f"- 推荐的模型类型: {model_type.upper()}\n")
    f.write(f"- 变量转换: {'对数转换' if transform_type == 'log' else '不转换'}\n")
    f.write(f"- 选择原因: {reason}\n\n")

# 准备建模数据
all_predictors = iv_list + control_vars
model_data = data[[dv_col] + all_predictors].copy()
model_data = model_data.dropna()  # 移除有缺失值的行
print(f"预处理后的数据: {len(model_data)} 行 × {len(model_data.columns)} 列")

# 处理因变量转换
transformed_dv = dv_col
if transform_type == "log":
    # 检查零值和负值
    min_value = model_data[dv_col].min()
    if min_value <= 0:
        print(f"警告: {dv_col} 的最小值为 {min_value}，包含零值或负值")
        print("将使用log(1+x)转换")
        model_data[f'log_{dv_col}'] = np.log1p(model_data[dv_col])
    else:
        model_data[f'log_{dv_col}'] = np.log(model_data[dv_col])
    transformed_dv = f'log_{dv_col}'
    print(f"已创建对数转换变量: {transformed_dv}")
else:
    print(f"使用原始变量: {transformed_dv}")

# 构建公式并显示
formula = f"{transformed_dv} ~ " + " + ".join(all_predictors)
print(f"回归公式:")
print(formula)

# 检查多重共线性
if len(all_predictors) > 1:
    X = model_data[all_predictors]
    vif_data = pd.DataFrame()
    vif_data["变量"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    print("\n多重共线性检验 (VIF):")
    print("VIF > 10 表示可能存在严重的多重共线性问题")
    print(vif_data.sort_values("VIF", ascending=False))

    with open(result_file, 'a', encoding='utf-8') as f:
        f.write("## 多重共线性检验 (VIF)\n\n")
        f.write("VIF > 10 表示可能存在严重的多重共线性问题\n\n")
        f.write("| 变量 | VIF |\n")
        f.write("|------|------|\n")
        for index, row in vif_data.iterrows():
            f.write(f"| {row['变量']} | {row['VIF']:.4f} |\n")
        f.write("\n")



数据加载完成，共 605 行和 67 列
从IV.txt加载了 18 个自变量
将使用的控制变量: author_followers_cnt, author_friends_cnt
已确认因变量'interaction_repost_cnt'存在于数据中
已确认所有控制变量存在于数据中
已创建结果文件: 生成结果/social_media/repost-模型结果.txt
数据预览:
                    id  title  \
0  1909480340126580992    NaN   
1  1909020699747886080    NaN   
2  1908203431765881088    NaN   
3  1908194727209385984    NaN   
4  1908053656731087104    NaN   

                                             content  url lang  \
0  Metal Shoe Rack\nUpto 24% off\nLink for you: h...  NaN   en   
1  @emob_ @DamnNearWhite @Tr3sMilagros @MyFirstKi...  NaN   en   
2  J.T. Foote Adjustable Shoe Trees - Plastic \n🛠...  NaN   en   
3  EXVITO Metal Shoe Rack for Home – Adjustable &...  NaN   en   
4  Closet Shoe Organizer for 24 Pairs with Adjust...  NaN   en   

                     publish_time publish_time_date publish_time_time  \
0  Tue Apr 08 05:36:04 +0000 2025        2025-04-08          05:36:04   
1  Sun Apr 06 23:09:37 +0000 2025        2025-04-06          23:0

In [15]:
print(f"开始拟合{model_type.upper()}模型...")

if model_type == "tobit":
    # 尝试不同的导入方式
    try:
        # 尝试从truncated_model模块导入
        from statsmodels.discrete.truncated_model import Tobit
        
        # 准备数据
        y = model_data[transformed_dv].values
        X = sm.add_constant(model_data[all_predictors])
        
        print("使用statsmodels.discrete.truncated_model中的Tobit模型...")
        tobit_model = Tobit(y, X, left=0)
        results = tobit_model.fit(method='powell', disp=0, maxiter=10000)
        
    except (ImportError, AttributeError):
        # 如果上述导入失败，使用censored_model模块
        try:
            from statsmodels.regression.censored_model import Tobit
            
            # 准备数据
            y = model_data[transformed_dv].values
            X = sm.add_constant(model_data[all_predictors])
            
            print("使用statsmodels.regression.censored_model中的Tobit模型...")
            tobit_model = Tobit(y, X, left=0)
            results = tobit_model.fit(method='powell', disp=0, maxiter=10000)
            
        except (ImportError, AttributeError):
            # 如果上述方法都失败，使用改进的自定义实现
            print("使用改进的自定义Tobit模型实现...")
            from scipy.stats import norm
            from scipy import optimize
            from statsmodels.regression.linear_model import OLS
            from statsmodels.base.model import GenericLikelihoodModel
            
            class TobitModel(GenericLikelihoodModel):
                def __init__(self, endog, exog, left=0, **kwds):
                    self.left = left
                    super(TobitModel, self).__init__(endog, exog, **kwds)
                
                def loglikeobs(self, params):
                    beta = params[:-1]
                    sigma = np.abs(params[-1])  # 确保sigma为正
                    
                    q = self.endog
                    x = self.exog
                    
                    # 计算条件期望
                    mu = np.dot(x, beta)
                    
                    # 分别计算截尾和非截尾值的对数似然
                    censored_mask = (q <= self.left)
                    z = (self.left - mu) / sigma
                    
                    ll_censored = censored_mask * norm.logcdf(z)
                    
                    non_censored_mask = ~censored_mask
                    ll_non_censored = non_censored_mask * (
                        -np.log(sigma) + 
                        norm.logpdf((q - mu) / sigma)
                    )
                    
                    # 处理可能的数值问题
                    result = ll_censored + ll_non_censored
                    # 替换无效值
                    result = np.where(np.isnan(result) | np.isinf(result), -1e10, result)
                    
                    return result
                
                def nloglikeobs(self, params):
                    """负对数似然"""
                    return -self.loglikeobs(params)
                
                def fit(self, start_params=None, method='bfgs', maxiter=50000, **kwds):
                    """添加更多优化方法选项和更好的初始值策略"""
                    if start_params is None:
                        # 使用OLS估计获取更稳定的初始值
                        ols_model = OLS(
                            np.where(self.endog <= self.left, self.left, self.endog),
                            self.exog
                        )
                        ols_res = ols_model.fit()
                        # 使用残差的标准差作为sigma的初始值
                        start_params = np.append(ols_res.params, np.std(ols_res.resid))
                    
                    # 添加更多优化选项
                    if 'options' not in kwds:
                        kwds['options'] = {}
                    kwds['options']['maxiter'] = maxiter
                    
                    # 添加容错设置
                    try:
                        return super(TobitModel, self).fit(
                            start_params=start_params,
                            method=method, 
                            **kwds
                        )
                    except Exception as e:
                        print(f"首次优化失败: {str(e)}，尝试备用方法...")
                        try:
                            # 尝试Powell方法
                            return super(TobitModel, self).fit(
                                start_params=start_params,
                                method='powell',
                                options={'maxiter': 100000, 'ftol': 1e-8, 'xtol': 1e-8},
                                **kwds
                            )
                        except:
                            print("所有优化方法失败，尝试最简单的优化设置...")
                            return super(TobitModel, self).fit(
                                start_params=start_params,
                                method='nm',
                                options={'maxiter': 100000},
                                **kwds
                            )
            
            # 准备数据 - 修改数据准备部分以避免AttributeError
            y = model_data[transformed_dv].values
            X_df = sm.add_constant(model_data[all_predictors])  # 保持DataFrame格式
            X_array = X_df.values  # 数组版本用于拟合
            X_columns = X_df.columns.tolist()  # 保存列名供后续使用
            
            # 拟合Tobit模型，尝试多种方法
            tobit_model = TobitModel(y, X_array, left=0)
            try:
                # 首先尝试Nelder-Mead方法
                results = tobit_model.fit(method='nm', disp=0, maxiter=50000)
            except:
                try:
                    # 如果失败，尝试BFGS方法
                    results = tobit_model.fit(method='bfgs', disp=0, maxiter=50000)
                except:
                    # 最后尝试Powell方法
                    results = tobit_model.fit(method='powell', disp=0, maxiter=50000)
    
    # 打印结果
    print("\nTobit模型结果:")
    print(results.summary())
    
else:  # 使用OLS模型 - 保持不变
    # 创建并拟合模型
    model = smf.ols(formula=formula, data=model_data)
    results = model.fit()
    
    # 显示结果摘要
    print("\nOLS回归结果:")
    print(results.summary())

# 保存结果
with open(result_file, 'a', encoding='utf-8') as f:
    f.write(f"## {model_type.upper()}回归结果\n\n")
    f.write(f"被解释变量: {transformed_dv}\n\n")
    f.write("\n")
    f.write(str(results.summary()))
    f.write("\n\n\n")

# 模型结果解读
print("\n模型解读:")

if model_type == "ols":
    # OLS模型特有的解读 - 保持不变
    print(f"- 模型解释力 (R²): {results.rsquared:.4f}")
    print(f"- 调整后的R²: {results.rsquared_adj:.4f}")
    print(f"- 模型整体显著性: F({results.df_model:.0f},{results.df_resid:.0f})={results.fvalue:.4f}, p={results.f_pvalue:.6f}")
    
    if results.f_pvalue < 0.05:
        print("  结论: 模型整体上显著")
    else:
        print("  结论: 模型整体上不显著")
    
    # 获取显著的变量
    p_values = results.pvalues[1:]  # 跳过截距
    var_names = all_predictors
    
else:  # Tobit模型 - 修改此部分以处理可能的NaN问题
    print("- Tobit模型统计量:")
    print(f"  - 对数似然值: {results.llf:.4f}")
    print(f"  - AIC: {results.aic:.4f}")
    print(f"  - BIC: {results.bic:.4f}")
    
    # 安全获取Tobit模型的系数和p值
    if hasattr(results, 'pvalues') and not np.all(np.isnan(results.pvalues)):
        p_values = results.pvalues[1:-1]  # 跳过截距和sigma
    else:
        print("警告: p值计算失败，将使用系数作为显著性的近似指标")
        # 使用系数作为显著性的近似指标
        coefs = results.params[1:-1]
        se = np.ones_like(coefs) * 0.05  # 假设标准误为0.05
        p_values = 2 * (1 - norm.cdf(np.abs(coefs / se)))
    
    # 在自定义Tobit中X_columns已定义，在statsmodels版本中需要定义
    if 'X_columns' not in locals():
        X_columns = model_data[all_predictors].columns.tolist()
        X_columns = ['const'] + X_columns
    
    var_names = X_columns[1:-1] if model_type == "tobit" else X_columns[1:]  # 适应不同情况

# 显著变量分析（对两种模型通用）
significant_mask = p_values < 0.05
significant_vars = [var for var, sig in zip(var_names, significant_mask) if sig]
print(f"\n显著的变量 (p < 0.05): {len(significant_vars)}/{len(var_names)}")

iv_significant = [var for var in significant_vars if var in iv_list]
ctrl_significant = [var for var in significant_vars if var in control_vars]

# 显示显著的自变量
print(f"\n显著的自变量: {len(iv_significant)}/{len(iv_list)}")
for var in iv_significant:
    idx = var_names.index(var)
    coef = results.params[idx + 1]  # +1 to skip intercept
    p = p_values[idx]
    print(f"- {var}: 系数={coef:.4f}, p={p:.6f}")

# 显示显著的控制变量
print(f"\n显著的控制变量: {len(ctrl_significant)}/{len(control_vars)}")
for var in ctrl_significant:
    idx = var_names.index(var)
    coef = results.params[idx + 1]  # +1 to skip intercept
    p = p_values[idx]
    print(f"- {var}: 系数={coef:.4f}, p={p:.6f}")

# 保存结果
with open(result_file, 'a', encoding='utf-8') as f:
    f.write(f"## {model_type.upper()}回归结果\n\n")
    f.write(f"被解释变量: {transformed_dv}\n\n")
    f.write("\n")
    f.write(str(results.summary()))
    f.write("\n\n\n")

# 模型结果解读
print("\n模型解读:")

if model_type == "ols":
    # OLS模型特有的解读
    print(f"- 模型解释力 (R²): {results.rsquared:.4f}")
    print(f"- 调整后的R²: {results.rsquared_adj:.4f}")
    print(f"- 模型整体显著性: F({results.df_model:.0f},{results.df_resid:.0f})={results.fvalue:.4f}, p={results.f_pvalue:.6f}")
    
    if results.f_pvalue < 0.05:
        print("  结论: 模型整体上显著")
    else:
        print("  结论: 模型整体上不显著")
    
    # 获取显著的变量
    p_values = results.pvalues[1:]  # 跳过截距
    var_names = all_predictors
    
else:  # Tobit模型
    print("- Tobit模型统计量:")
    print(f"  - 对数似然值: {results.llf:.4f}")
    print(f"  - AIC: {results.aic:.4f}")
    print(f"  - BIC: {results.bic:.4f}")
    
    # Tobit模型中获取系数和p值
    p_values = results.pvalues[1:-1]  # 跳过截距和sigma
    var_names = list(X.columns[1:])  # 跳过常数项

# 显著变量分析（对两种模型通用）
significant_mask = p_values < 0.05
significant_vars = [var for var, sig in zip(var_names, significant_mask) if sig]
print(f"\n显著的变量 (p < 0.05): {len(significant_vars)}/{len(var_names)}")

iv_significant = [var for var in significant_vars if var in iv_list]
ctrl_significant = [var for var in significant_vars if var in control_vars]

# 显示显著的自变量
print(f"\n显著的自变量: {len(iv_significant)}/{len(iv_list)}")
for var in iv_significant:
    idx = var_names.index(var)
    coef = results.params[idx + 1]  # +1 to skip intercept
    p = p_values[idx]
    print(f"- {var}: 系数={coef:.4f}, p={p:.6f}")

# 显示显著的控制变量
print(f"\n显著的控制变量: {len(ctrl_significant)}/{len(control_vars)}")
for var in ctrl_significant:
    idx = var_names.index(var)
    coef = results.params[idx + 1]  # +1 to skip intercept
    p = p_values[idx]
    print(f"- {var}: 系数={coef:.4f}, p={p:.6f}")

# 保存解读结果
with open(result_file, 'a', encoding='utf-8') as f:
    f.write("## 模型结果解读\n\n")
    
    if model_type == "ols":
        f.write(f"- 模型解释力 (R²): {results.rsquared:.4f}\n")
        f.write(f"- 调整后的R²: {results.rsquared_adj:.4f}\n")
        f.write(f"- 模型整体显著性: F({results.df_model:.0f},{results.df_resid:.0f})={results.fvalue:.4f}, p={results.f_pvalue:.6f}\n")
        
        if results.f_pvalue < 0.05:
            f.write("  结论: 模型整体上显著\n\n")
        else:
            f.write("  结论: 模型整体上不显著\n\n")
    else:
        f.write("- Tobit模型统计量:\n")
        f.write(f"  - 对数似然值: {results.llf:.4f}\n")
        f.write(f"  - AIC: {results.aic:.4f}\n")
        f.write(f"  - BIC: {results.bic:.4f}\n\n")
    
    f.write(f"显著的变量 (p < 0.05): {len(significant_vars)}/{len(var_names)}\n\n")
    
    f.write(f"显著的自变量: {len(iv_significant)}/{len(iv_list)}\n")
    for var in iv_significant:
        idx = var_names.index(var)
        coef = results.params[idx + 1]
        p = p_values[idx]
        f.write(f"- {var}: 系数={coef:.4f}, p={p:.6f}\n")
    
    f.write(f"\n显著的控制变量: {len(ctrl_significant)}/{len(control_vars)}\n")
    for var in ctrl_significant:
        idx = var_names.index(var)
        coef = results.params[idx + 1]
        p = p_values[idx]
        f.write(f"- {var}: 系数={coef:.4f}, p={p:.6f}\n")

print(f"\n所有分析结果已保存到 {result_file}")


开始拟合TOBIT模型...
使用改进的自定义Tobit模型实现...


C:\Users\admin\anaconda3\envs\Lynx\lib\site-packages\statsmodels\base\optimizer.py:19: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method nm is: xtol, ftol, maxfun. The list of unsupported keyword arguments passed include: options. After release 0.14, this will raise.
  warnings.warn(



Tobit模型结果:
                              TobitModel Results                              
Dep. Variable:                      y   Log-Likelihood:                -376.47
Model:                     TobitModel   AIC:                             796.9
Method:            Maximum Likelihood   BIC:                             893.9
Date:                Tue, 15 Apr 2025                                         
Time:                        18:46:12                                         
No. Observations:                 605                                         
Df Residuals:                     584                                         
Df Model:                          20                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0062        nan        nan        nan         nan         nan
x1            -0.4776        nan        

C:\Users\admin\anaconda3\envs\Lynx\lib\site-packages\statsmodels\base\model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
C:\Users\admin\anaconda3\envs\Lynx\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\admin\anaconda3\envs\Lynx\lib\site-packages\statsmodels\base\model.py:2748: UserWarning: df_model + k_constant + k_extra differs from k_params
  warnings.warn("df_model + k_constant + k_extra "
C:\Users\admin\anaconda3\envs\Lynx\lib\site-packages\statsmodels\base\model.py:2752: UserWarning: df_resid differs from nobs - k_params
  warnings.warn("df_resid differs from nobs - k_params")


In [16]:
# 归一化显著系数并输出结果
def normalize_significant_coefficients(results, iv_list, control_vars=None, model_type="ols", alpha=0.05):
    """
    归一化显著变量的系数，删除NaN值，按绝对值大小排序并输出结果
    
    参数:
    - results: 回归结果对象
    - iv_list: 自变量列表
    - control_vars: 控制变量列表，默认为None
    - model_type: 模型类型，"ols"或"tobit"
    - alpha: 显著性水平，默认0.05
    """
    if control_vars is None:
        control_vars = []
    
    all_vars = iv_list + control_vars
    
    # 获取系数和p值 - 需要考虑不同模型类型
    if model_type.lower() == "ols":
        # OLS模型 - 跳过截距
        coefs = results.params[1:len(all_vars)+1]
        p_values = results.pvalues[1:len(all_vars)+1]
    else:
        # Tobit模型 - 跳过截距和sigma
        coefs = results.params[1:len(all_vars)+1]
        p_values = results.pvalues[1:len(all_vars)+1]
    
    # 创建DataFrame
    coef_df = pd.DataFrame({
        '变量': all_vars,
        '系数': coefs,
        'p值': p_values,
        '变量类型': ['自变量' if var in iv_list else '控制变量' for var in all_vars]
    })
    
    # 过滤掉NaN系数和非显著的变量
    valid_coef_df = coef_df.dropna(subset=['系数'])
    sig_coef_df = valid_coef_df[valid_coef_df['p值'] < alpha].copy()
    
    # 如果没有显著变量，则报告所有有效变量
    if len(sig_coef_df) == 0:
        print("注意: 没有显著的变量 (p < 0.05)，将使用所有非NaN系数变量")
        sig_coef_df = valid_coef_df.copy()
    
    # 如果没有有效系数，退出
    if len(sig_coef_df) == 0:
        print("警告: 没有有效的系数，无法进行归一化")
        return None
    
    # 计算系数的绝对值
    sig_coef_df['系数绝对值'] = sig_coef_df['系数'].abs()
    
    # 归一化系数
    total_abs = sig_coef_df['系数绝对值'].sum()
    sig_coef_df['归一化系数'] = sig_coef_df['系数'] / total_abs
    sig_coef_df['归一化系数绝对值'] = sig_coef_df['系数绝对值'] / total_abs
    sig_coef_df['归一化系数百分比'] = sig_coef_df['归一化系数绝对值'] * 100
    
    # 按系数绝对值排序
    sig_coef_df = sig_coef_df.sort_values('系数绝对值', ascending=False)
    
    # 显示结果
    print("\n归一化后的系数 (按绝对值排序):")
    print("-----------------------------------")
    significant_str = "显著 (p<0.05)" if len(sig_coef_df[sig_coef_df['p值'] < alpha]) > 0 else "无显著变量"
    print(f"总计 {len(sig_coef_df)} 个有效变量, {significant_str}")
    print("-----------------------------------")
    
    # 先显示自变量，再显示控制变量
    for var_type in ['自变量', '控制变量']:
        type_df = sig_coef_df[sig_coef_df['变量类型'] == var_type]
        if len(type_df) > 0:
            print(f"\n{var_type}:")
            for _, row in type_df.iterrows():
                sig_mark = "*" if row['p值'] < alpha else ""
                print(f"{row['变量']}: {row['系数']:.4f} (归一化: {row['归一化系数']:.4f}, {row['归一化系数百分比']:.2f}%){sig_mark}")
    
    print("-----------------------------------")
    print("* 标记表示在0.05水平上显著")
    
    # 保存到文件
    output_file = os.path.join(folder_name, "Repost-归一化的模型结果.txt")
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(f"# {model_type.upper()} 模型归一化后的系数 (按绝对值排序)\n\n")
        f.write("-----------------------------------\n")
        f.write(f"总计 {len(sig_coef_df)} 个有效变量, {significant_str}\n")
        f.write("-----------------------------------\n\n")
        
        # 添加总表
        f.write("| 变量 | 变量类型 | 系数 | p值 | 归一化系数 | 归一化百分比 | 显著性 |\n")
        f.write("|------|---------|------|------|------------|------------|--------|\n")
        
        for _, row in sig_coef_df.iterrows():
            sig = "是" if row['p值'] < alpha else "否"
            f.write(f"| {row['变量']} | {row['变量类型']} | {row['系数']:.4f} | {row['p值']:.4f} | {row['归一化系数']:.4f} | {row['归一化系数百分比']:.2f}% | {sig} |\n")
        
        f.write("\n\n## 详细分析\n\n")
        
        # 先显示自变量，再显示控制变量
        for var_type in ['自变量', '控制变量']:
            type_df = sig_coef_df[sig_coef_df['变量类型'] == var_type]
            if len(type_df) > 0:
                f.write(f"### {var_type}\n\n")
                
                for _, row in type_df.iterrows():
                    sig_symbol = "*" if row['p值'] < alpha else ""
                    direction = "正向" if row['系数'] > 0 else "负向"
                    f.write(f"#### {row['变量']}{sig_symbol}\n\n")
                    f.write(f"- 系数: {row['系数']:.4f}\n")
                    f.write(f"- p值: {row['p值']:.4f}\n")
                    f.write(f"- 影响方向: {direction}\n")
                    f.write(f"- 归一化系数: {row['归一化系数']:.4f}\n")
                    f.write(f"- 归一化百分比: {row['归一化系数百分比']:.2f}%\n")
                    f.write(f"- 是否显著: {'是' if row['p值'] < alpha else '否'}\n\n")
        
        f.write("-----------------------------------\n")
        f.write("* 标记表示在0.05水平上显著\n")
    
    print(f"\n结果已保存到 {output_file}")
    
    return sig_coef_df
# 调用归一化函数
print("\n开始归一化系数分析...")
normalized_coefs = normalize_significant_coefficients(
    results=results,
    iv_list=iv_list,
    control_vars=control_vars,
    model_type=model_type,
    alpha=0.05
)



开始归一化系数分析...
注意: 没有显著的变量 (p < 0.05)，将使用所有非NaN系数变量

归一化后的系数 (按绝对值排序):
-----------------------------------
总计 20 个有效变量, 无显著变量
-----------------------------------

自变量:
安全性需求表达: -0.4776 (归一化: -0.1825, 18.25%)
品牌声誉表达: -0.3071 (归一化: -0.1173, 11.73%)
便携性表达: -0.2699 (归一化: -0.1032, 10.32%)
儿童友好设计表达: -0.2253 (归一化: -0.0861, 8.61%)
层高可调节性表达: -0.1915 (归一化: -0.0732, 7.32%)
季节适应性表达: -0.1740 (归一化: -0.0665, 6.65%)
价格表达: 0.1713 (归一化: 0.0655, 6.55%)
空间效率表达: -0.1491 (归一化: -0.0570, 5.70%)
环保材质偏好: 0.1470 (归一化: 0.0562, 5.62%)
组合灵活性表达: 0.1302 (归一化: 0.0498, 4.98%)
耐用性表达: -0.1068 (归一化: -0.0408, 4.08%)
承重能力表达: -0.0998 (归一化: -0.0381, 3.81%)
多功能性表达: -0.0902 (归一化: -0.0345, 3.45%)
组装便捷性表达: -0.0402 (归一化: -0.0154, 1.54%)
创新功能表达: -0.0277 (归一化: -0.0106, 1.06%)
颜色选择表达: -0.0052 (归一化: -0.0020, 0.20%)
防尘功能表达: 0.0037 (归一化: 0.0014, 0.14%)
设计美感表达: -0.0003 (归一化: -0.0001, 0.01%)

控制变量:
author_friends_cnt: 0.0001 (归一化: 0.0000, 0.00%)
author_followers_cnt: -0.0000 (归一化: -0.0000, 0.00%)
-----------------------------------
* 标记表示在

### 分享分析

In [17]:
# 加载数据
data_file = os.path.join(folder_name, "数据变量语义匹配二元赋值结果.xlsx")
data = pd.read_excel(data_file)
print(f"数据加载完成，共 {len(data)} 行和 {len(data.columns)} 列")

# 修改IV文件路径
iv_file = os.path.join(folder_name, 'IV.txt')
with open(iv_file, 'r', encoding='utf-8') as f:
    iv_list = f.read().splitlines()
    iv_list = [iv.strip() for iv in iv_list if iv.strip()]  # 移除空行
print(f"从IV.txt加载了 {len(iv_list)} 个自变量")

# 定义因变量和控制变量
dv_col = 'interaction_share_cnt'
control_vars = ['author_followers_cnt', 'author_friends_cnt']
print(f"将使用的控制变量: {', '.join(control_vars)}")

# 检查DV列是否存在
if dv_col not in data.columns:
    print(f"错误: 数据中不存在列 '{dv_col}'")
    print("数据包含的列:", data.columns.tolist()[:10], "...")
    raise ValueError(f"找不到DV列: {dv_col}")
else:
    print(f"已确认因变量'{dv_col}'存在于数据中")

# 检查控制变量是否存在
missing_controls = [var for var in control_vars if var not in data.columns]
if missing_controls:
    print(f"错误: 以下控制变量在数据中不存在: {missing_controls}")
    raise ValueError("缺少必要的控制变量")
else:
    print("已确认所有控制变量存在于数据中")

# 检查IV列是否都存在
missing_ivs = [iv for iv in iv_list if iv not in data.columns]
if missing_ivs:
    print(f"警告: 以下IV在数据中不存在: {missing_ivs}")
    iv_list = [iv for iv in iv_list if iv in data.columns]
    print(f"将使用剩余的 {len(iv_list)} 个IV进行建模")

if not iv_list:
    raise ValueError("没有可用的IV变量进行建模")

# 修改结果文件路径
result_file = os.path.join(folder_name, "share-模型结果.txt")
with open(result_file, 'w', encoding='utf-8') as f:
    f.write("# 模型分析结果\n\n")
print(f"已创建结果文件: {result_file}")

# 显示数据前几行
print("数据预览:")
print(data.head())

# 分析DV的分布
def analyze_distribution(data, col):
    """
    分析变量分布并返回基本统计量和模型推荐
    """
    # 提取非空数据
    valid_data = data[col].dropna()
    
    # 基本统计量
    stats_dict = {
        "样本量": len(valid_data),
        "零值数量": (valid_data == 0).sum(),
        "零值比例": (valid_data == 0).mean() * 100,
        "均值": valid_data.mean(),
        "中位数": valid_data.median(),
        "标准差": valid_data.std(),
        "最小值": valid_data.min(),
        "最大值": valid_data.max(),
        "偏度": skew(valid_data),
        "峰度": kurtosis(valid_data)
    }

    # 模型选择逻辑 - 根据零值比例和分布偏度
    zero_inflated = stats_dict["零值比例"] > 20  # 如果零值超过20%，使用Tobit
    high_skew = abs(stats_dict["偏度"]) > 1.0   # 如果偏度较大，使用对数转换
    
    model_type = "ols"  # 默认模型类型
    transform_type = "none"  # 默认不转换
    reason = ""
    
    if zero_inflated:
        model_type = "tobit"
        reason += f"数据中零值比例为{stats_dict['零值比例']:.2f}%，超过20%，使用Tobit模型。"
        
        if high_skew:
            transform_type = "log"
            reason += f"数据偏度为{stats_dict['偏度']:.2f}，表现为偏态分布，对因变量进行对数转换。"
    else:
        model_type = "ols"
        reason += f"数据中零值比例为{stats_dict['零值比例']:.2f}%，不超过20%，使用OLS模型。"
        
        if high_skew:
            transform_type = "log"
            reason += f"数据偏度为{stats_dict['偏度']:.2f}，表现为偏态分布，对因变量进行对数转换。"
    
    return model_type, transform_type, reason, stats_dict

# 对DV进行分布分析
model_type, transform_type, reason, dv_stats = analyze_distribution(data, dv_col)

# 显示分析结果
print(f"\n## {dv_col} 分布分析结果:")
for key, value in dv_stats.items():
    if isinstance(value, float):
        print(f"- {key}: {value:.4f}")
    else:
        print(f"- {key}: {value}")

print(f"\n## 推荐的模型: {model_type.upper()}")
print(f"- 变量转换: {'对数转换' if transform_type == 'log' else '不转换'}")
print(f"- 原因: {reason}")

# 将结果保存到文件
with open(result_file, 'a', encoding='utf-8') as f:
    f.write(f"## {dv_col} 分布分析\n\n")
    for key, value in dv_stats.items():
        if isinstance(value, float):
            f.write(f"- {key}: {value:.4f}\n")
        else:
            f.write(f"- {key}: {value}\n")
    
    f.write(f"\n## 模型选择\n\n")
    f.write(f"- 推荐的模型类型: {model_type.upper()}\n")
    f.write(f"- 变量转换: {'对数转换' if transform_type == 'log' else '不转换'}\n")
    f.write(f"- 选择原因: {reason}\n\n")

# 准备建模数据
all_predictors = iv_list + control_vars
model_data = data[[dv_col] + all_predictors].copy()
model_data = model_data.dropna()  # 移除有缺失值的行
print(f"预处理后的数据: {len(model_data)} 行 × {len(model_data.columns)} 列")

# 处理因变量转换
transformed_dv = dv_col
if transform_type == "log":
    # 检查零值和负值
    min_value = model_data[dv_col].min()
    if min_value <= 0:
        print(f"警告: {dv_col} 的最小值为 {min_value}，包含零值或负值")
        print("将使用log(1+x)转换")
        model_data[f'log_{dv_col}'] = np.log1p(model_data[dv_col])
    else:
        model_data[f'log_{dv_col}'] = np.log(model_data[dv_col])
    transformed_dv = f'log_{dv_col}'
    print(f"已创建对数转换变量: {transformed_dv}")
else:
    print(f"使用原始变量: {transformed_dv}")

# 构建公式并显示
formula = f"{transformed_dv} ~ " + " + ".join(all_predictors)
print(f"回归公式:")
print(formula)

# 检查多重共线性
if len(all_predictors) > 1:
    X = model_data[all_predictors]
    vif_data = pd.DataFrame()
    vif_data["变量"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    print("\n多重共线性检验 (VIF):")
    print("VIF > 10 表示可能存在严重的多重共线性问题")
    print(vif_data.sort_values("VIF", ascending=False))

    with open(result_file, 'a', encoding='utf-8') as f:
        f.write("## 多重共线性检验 (VIF)\n\n")
        f.write("VIF > 10 表示可能存在严重的多重共线性问题\n\n")
        f.write("| 变量 | VIF |\n")
        f.write("|------|------|\n")
        for index, row in vif_data.iterrows():
            f.write(f"| {row['变量']} | {row['VIF']:.4f} |\n")
        f.write("\n")



数据加载完成，共 605 行和 67 列
从IV.txt加载了 18 个自变量
将使用的控制变量: author_followers_cnt, author_friends_cnt
已确认因变量'interaction_share_cnt'存在于数据中
已确认所有控制变量存在于数据中
已创建结果文件: 生成结果/social_media/share-模型结果.txt
数据预览:
                    id  title  \
0  1909480340126580992    NaN   
1  1909020699747886080    NaN   
2  1908203431765881088    NaN   
3  1908194727209385984    NaN   
4  1908053656731087104    NaN   

                                             content  url lang  \
0  Metal Shoe Rack\nUpto 24% off\nLink for you: h...  NaN   en   
1  @emob_ @DamnNearWhite @Tr3sMilagros @MyFirstKi...  NaN   en   
2  J.T. Foote Adjustable Shoe Trees - Plastic \n🛠...  NaN   en   
3  EXVITO Metal Shoe Rack for Home – Adjustable &...  NaN   en   
4  Closet Shoe Organizer for 24 Pairs with Adjust...  NaN   en   

                     publish_time publish_time_date publish_time_time  \
0  Tue Apr 08 05:36:04 +0000 2025        2025-04-08          05:36:04   
1  Sun Apr 06 23:09:37 +0000 2025        2025-04-06          23:09:

In [18]:
print(f"开始拟合{model_type.upper()}模型...")

if model_type == "tobit":
    # 尝试不同的导入方式
    try:
        # 尝试从truncated_model模块导入
        from statsmodels.discrete.truncated_model import Tobit
        
        # 准备数据
        y = model_data[transformed_dv].values
        X = sm.add_constant(model_data[all_predictors])
        
        print("使用statsmodels.discrete.truncated_model中的Tobit模型...")
        tobit_model = Tobit(y, X, left=0)
        results = tobit_model.fit(method='powell', disp=0, maxiter=10000)
        
    except (ImportError, AttributeError):
        # 如果上述导入失败，使用censored_model模块
        try:
            from statsmodels.regression.censored_model import Tobit
            
            # 准备数据
            y = model_data[transformed_dv].values
            X = sm.add_constant(model_data[all_predictors])
            
            print("使用statsmodels.regression.censored_model中的Tobit模型...")
            tobit_model = Tobit(y, X, left=0)
            results = tobit_model.fit(method='powell', disp=0, maxiter=10000)
            
        except (ImportError, AttributeError):
            # 如果上述方法都失败，使用改进的自定义实现
            print("使用改进的自定义Tobit模型实现...")
            from scipy.stats import norm
            from scipy import optimize
            from statsmodels.regression.linear_model import OLS
            from statsmodels.base.model import GenericLikelihoodModel
            
            class TobitModel(GenericLikelihoodModel):
                def __init__(self, endog, exog, left=0, **kwds):
                    self.left = left
                    super(TobitModel, self).__init__(endog, exog, **kwds)
                
                def loglikeobs(self, params):
                    beta = params[:-1]
                    sigma = np.abs(params[-1])  # 确保sigma为正
                    
                    q = self.endog
                    x = self.exog
                    
                    # 计算条件期望
                    mu = np.dot(x, beta)
                    
                    # 分别计算截尾和非截尾值的对数似然
                    censored_mask = (q <= self.left)
                    z = (self.left - mu) / sigma
                    
                    ll_censored = censored_mask * norm.logcdf(z)
                    
                    non_censored_mask = ~censored_mask
                    ll_non_censored = non_censored_mask * (
                        -np.log(sigma) + 
                        norm.logpdf((q - mu) / sigma)
                    )
                    
                    # 处理可能的数值问题
                    result = ll_censored + ll_non_censored
                    # 替换无效值
                    result = np.where(np.isnan(result) | np.isinf(result), -1e10, result)
                    
                    return result
                
                def nloglikeobs(self, params):
                    """负对数似然"""
                    return -self.loglikeobs(params)
                
                def fit(self, start_params=None, method='bfgs', maxiter=50000, **kwds):
                    """添加更多优化方法选项和更好的初始值策略"""
                    if start_params is None:
                        # 使用OLS估计获取更稳定的初始值
                        ols_model = OLS(
                            np.where(self.endog <= self.left, self.left, self.endog),
                            self.exog
                        )
                        ols_res = ols_model.fit()
                        # 使用残差的标准差作为sigma的初始值
                        start_params = np.append(ols_res.params, np.std(ols_res.resid))
                    
                    # 添加更多优化选项
                    if 'options' not in kwds:
                        kwds['options'] = {}
                    kwds['options']['maxiter'] = maxiter
                    
                    # 添加容错设置
                    try:
                        return super(TobitModel, self).fit(
                            start_params=start_params,
                            method=method, 
                            **kwds
                        )
                    except Exception as e:
                        print(f"首次优化失败: {str(e)}，尝试备用方法...")
                        try:
                            # 尝试Powell方法
                            return super(TobitModel, self).fit(
                                start_params=start_params,
                                method='powell',
                                options={'maxiter': 100000, 'ftol': 1e-8, 'xtol': 1e-8},
                                **kwds
                            )
                        except:
                            print("所有优化方法失败，尝试最简单的优化设置...")
                            return super(TobitModel, self).fit(
                                start_params=start_params,
                                method='nm',
                                options={'maxiter': 100000},
                                **kwds
                            )
            
            # 准备数据 - 修改数据准备部分以避免AttributeError
            y = model_data[transformed_dv].values
            X_df = sm.add_constant(model_data[all_predictors])  # 保持DataFrame格式
            X_array = X_df.values  # 数组版本用于拟合
            X_columns = X_df.columns.tolist()  # 保存列名供后续使用
            
            # 拟合Tobit模型，尝试多种方法
            tobit_model = TobitModel(y, X_array, left=0)
            try:
                # 首先尝试Nelder-Mead方法
                results = tobit_model.fit(method='nm', disp=0, maxiter=50000)
            except:
                try:
                    # 如果失败，尝试BFGS方法
                    results = tobit_model.fit(method='bfgs', disp=0, maxiter=50000)
                except:
                    # 最后尝试Powell方法
                    results = tobit_model.fit(method='powell', disp=0, maxiter=50000)
    
    # 打印结果
    print("\nTobit模型结果:")
    print(results.summary())
    
else:  # 使用OLS模型 - 保持不变
    # 创建并拟合模型
    model = smf.ols(formula=formula, data=model_data)
    results = model.fit()
    
    # 显示结果摘要
    print("\nOLS回归结果:")
    print(results.summary())

# 保存结果
with open(result_file, 'a', encoding='utf-8') as f:
    f.write(f"## {model_type.upper()}回归结果\n\n")
    f.write(f"被解释变量: {transformed_dv}\n\n")
    f.write("\n")
    f.write(str(results.summary()))
    f.write("\n\n\n")

# 模型结果解读
print("\n模型解读:")

if model_type == "ols":
    # OLS模型特有的解读 - 保持不变
    print(f"- 模型解释力 (R²): {results.rsquared:.4f}")
    print(f"- 调整后的R²: {results.rsquared_adj:.4f}")
    print(f"- 模型整体显著性: F({results.df_model:.0f},{results.df_resid:.0f})={results.fvalue:.4f}, p={results.f_pvalue:.6f}")
    
    if results.f_pvalue < 0.05:
        print("  结论: 模型整体上显著")
    else:
        print("  结论: 模型整体上不显著")
    
    # 获取显著的变量
    p_values = results.pvalues[1:]  # 跳过截距
    var_names = all_predictors
    
else:  # Tobit模型 - 修改此部分以处理可能的NaN问题
    print("- Tobit模型统计量:")
    print(f"  - 对数似然值: {results.llf:.4f}")
    print(f"  - AIC: {results.aic:.4f}")
    print(f"  - BIC: {results.bic:.4f}")
    
    # 安全获取Tobit模型的系数和p值
    if hasattr(results, 'pvalues') and not np.all(np.isnan(results.pvalues)):
        p_values = results.pvalues[1:-1]  # 跳过截距和sigma
    else:
        print("警告: p值计算失败，将使用系数作为显著性的近似指标")
        # 使用系数作为显著性的近似指标
        coefs = results.params[1:-1]
        se = np.ones_like(coefs) * 0.05  # 假设标准误为0.05
        p_values = 2 * (1 - norm.cdf(np.abs(coefs / se)))
    
    # 在自定义Tobit中X_columns已定义，在statsmodels版本中需要定义
    if 'X_columns' not in locals():
        X_columns = model_data[all_predictors].columns.tolist()
        X_columns = ['const'] + X_columns
    
    var_names = X_columns[1:-1] if model_type == "tobit" else X_columns[1:]  # 适应不同情况

# 显著变量分析（对两种模型通用）
significant_mask = p_values < 0.05
significant_vars = [var for var, sig in zip(var_names, significant_mask) if sig]
print(f"\n显著的变量 (p < 0.05): {len(significant_vars)}/{len(var_names)}")

iv_significant = [var for var in significant_vars if var in iv_list]
ctrl_significant = [var for var in significant_vars if var in control_vars]

# 显示显著的自变量
print(f"\n显著的自变量: {len(iv_significant)}/{len(iv_list)}")
for var in iv_significant:
    idx = var_names.index(var)
    coef = results.params[idx + 1]  # +1 to skip intercept
    p = p_values[idx]
    print(f"- {var}: 系数={coef:.4f}, p={p:.6f}")

# 显示显著的控制变量
print(f"\n显著的控制变量: {len(ctrl_significant)}/{len(control_vars)}")
for var in ctrl_significant:
    idx = var_names.index(var)
    coef = results.params[idx + 1]  # +1 to skip intercept
    p = p_values[idx]
    print(f"- {var}: 系数={coef:.4f}, p={p:.6f}")

# 保存结果
with open(result_file, 'a', encoding='utf-8') as f:
    f.write(f"## {model_type.upper()}回归结果\n\n")
    f.write(f"被解释变量: {transformed_dv}\n\n")
    f.write("\n")
    f.write(str(results.summary()))
    f.write("\n\n\n")

# 模型结果解读
print("\n模型解读:")

if model_type == "ols":
    # OLS模型特有的解读
    print(f"- 模型解释力 (R²): {results.rsquared:.4f}")
    print(f"- 调整后的R²: {results.rsquared_adj:.4f}")
    print(f"- 模型整体显著性: F({results.df_model:.0f},{results.df_resid:.0f})={results.fvalue:.4f}, p={results.f_pvalue:.6f}")
    
    if results.f_pvalue < 0.05:
        print("  结论: 模型整体上显著")
    else:
        print("  结论: 模型整体上不显著")
    
    # 获取显著的变量
    p_values = results.pvalues[1:]  # 跳过截距
    var_names = all_predictors
    
else:  # Tobit模型
    print("- Tobit模型统计量:")
    print(f"  - 对数似然值: {results.llf:.4f}")
    print(f"  - AIC: {results.aic:.4f}")
    print(f"  - BIC: {results.bic:.4f}")
    
    # Tobit模型中获取系数和p值
    p_values = results.pvalues[1:-1]  # 跳过截距和sigma
    var_names = list(X.columns[1:])  # 跳过常数项

# 显著变量分析（对两种模型通用）
significant_mask = p_values < 0.05
significant_vars = [var for var, sig in zip(var_names, significant_mask) if sig]
print(f"\n显著的变量 (p < 0.05): {len(significant_vars)}/{len(var_names)}")

iv_significant = [var for var in significant_vars if var in iv_list]
ctrl_significant = [var for var in significant_vars if var in control_vars]

# 显示显著的自变量
print(f"\n显著的自变量: {len(iv_significant)}/{len(iv_list)}")
for var in iv_significant:
    idx = var_names.index(var)
    coef = results.params[idx + 1]  # +1 to skip intercept
    p = p_values[idx]
    print(f"- {var}: 系数={coef:.4f}, p={p:.6f}")

# 显示显著的控制变量
print(f"\n显著的控制变量: {len(ctrl_significant)}/{len(control_vars)}")
for var in ctrl_significant:
    idx = var_names.index(var)
    coef = results.params[idx + 1]  # +1 to skip intercept
    p = p_values[idx]
    print(f"- {var}: 系数={coef:.4f}, p={p:.6f}")

# 保存解读结果
with open(result_file, 'a', encoding='utf-8') as f:
    f.write("## 模型结果解读\n\n")
    
    if model_type == "ols":
        f.write(f"- 模型解释力 (R²): {results.rsquared:.4f}\n")
        f.write(f"- 调整后的R²: {results.rsquared_adj:.4f}\n")
        f.write(f"- 模型整体显著性: F({results.df_model:.0f},{results.df_resid:.0f})={results.fvalue:.4f}, p={results.f_pvalue:.6f}\n")
        
        if results.f_pvalue < 0.05:
            f.write("  结论: 模型整体上显著\n\n")
        else:
            f.write("  结论: 模型整体上不显著\n\n")
    else:
        f.write("- Tobit模型统计量:\n")
        f.write(f"  - 对数似然值: {results.llf:.4f}\n")
        f.write(f"  - AIC: {results.aic:.4f}\n")
        f.write(f"  - BIC: {results.bic:.4f}\n\n")
    
    f.write(f"显著的变量 (p < 0.05): {len(significant_vars)}/{len(var_names)}\n\n")
    
    f.write(f"显著的自变量: {len(iv_significant)}/{len(iv_list)}\n")
    for var in iv_significant:
        idx = var_names.index(var)
        coef = results.params[idx + 1]
        p = p_values[idx]
        f.write(f"- {var}: 系数={coef:.4f}, p={p:.6f}\n")
    
    f.write(f"\n显著的控制变量: {len(ctrl_significant)}/{len(control_vars)}\n")
    for var in ctrl_significant:
        idx = var_names.index(var)
        coef = results.params[idx + 1]
        p = p_values[idx]
        f.write(f"- {var}: 系数={coef:.4f}, p={p:.6f}\n")

print(f"\n所有分析结果已保存到 {result_file}")


开始拟合TOBIT模型...
使用改进的自定义Tobit模型实现...


C:\Users\admin\anaconda3\envs\Lynx\lib\site-packages\statsmodels\base\optimizer.py:19: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method nm is: xtol, ftol, maxfun. The list of unsupported keyword arguments passed include: options. After release 0.14, this will raise.
  warnings.warn(



Tobit模型结果:
                              TobitModel Results                              
Dep. Variable:                      y   Log-Likelihood:                -376.93
Model:                     TobitModel   AIC:                             797.9
Method:            Maximum Likelihood   BIC:                             894.8
Date:                Tue, 15 Apr 2025                                         
Time:                        18:46:13                                         
No. Observations:                 605                                         
Df Residuals:                     584                                         
Df Model:                          20                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       3.256e-05        nan        nan        nan         nan         nan
x1            -0.0127        nan        

C:\Users\admin\anaconda3\envs\Lynx\lib\site-packages\statsmodels\base\model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
C:\Users\admin\anaconda3\envs\Lynx\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\admin\anaconda3\envs\Lynx\lib\site-packages\statsmodels\base\model.py:2748: UserWarning: df_model + k_constant + k_extra differs from k_params
  warnings.warn("df_model + k_constant + k_extra "
C:\Users\admin\anaconda3\envs\Lynx\lib\site-packages\statsmodels\base\model.py:2752: UserWarning: df_resid differs from nobs - k_params
  warnings.warn("df_resid differs from nobs - k_params")


In [19]:
# 归一化显著系数并输出结果
def normalize_significant_coefficients(results, iv_list, control_vars=None, model_type="ols", alpha=0.05):
    """
    归一化显著变量的系数，删除NaN值，按绝对值大小排序并输出结果
    
    参数:
    - results: 回归结果对象
    - iv_list: 自变量列表
    - control_vars: 控制变量列表，默认为None
    - model_type: 模型类型，"ols"或"tobit"
    - alpha: 显著性水平，默认0.05
    """
    if control_vars is None:
        control_vars = []
    
    all_vars = iv_list + control_vars
    
    # 获取系数和p值 - 需要考虑不同模型类型
    if model_type.lower() == "ols":
        # OLS模型 - 跳过截距
        coefs = results.params[1:len(all_vars)+1]
        p_values = results.pvalues[1:len(all_vars)+1]
    else:
        # Tobit模型 - 跳过截距和sigma
        coefs = results.params[1:len(all_vars)+1]
        p_values = results.pvalues[1:len(all_vars)+1]
    
    # 创建DataFrame
    coef_df = pd.DataFrame({
        '变量': all_vars,
        '系数': coefs,
        'p值': p_values,
        '变量类型': ['自变量' if var in iv_list else '控制变量' for var in all_vars]
    })
    
    # 过滤掉NaN系数和非显著的变量
    valid_coef_df = coef_df.dropna(subset=['系数'])
    sig_coef_df = valid_coef_df[valid_coef_df['p值'] < alpha].copy()
    
    # 如果没有显著变量，则报告所有有效变量
    if len(sig_coef_df) == 0:
        print("注意: 没有显著的变量 (p < 0.05)，将使用所有非NaN系数变量")
        sig_coef_df = valid_coef_df.copy()
    
    # 如果没有有效系数，退出
    if len(sig_coef_df) == 0:
        print("警告: 没有有效的系数，无法进行归一化")
        return None
    
    # 计算系数的绝对值
    sig_coef_df['系数绝对值'] = sig_coef_df['系数'].abs()
    
    # 归一化系数
    total_abs = sig_coef_df['系数绝对值'].sum()
    sig_coef_df['归一化系数'] = sig_coef_df['系数'] / total_abs
    sig_coef_df['归一化系数绝对值'] = sig_coef_df['系数绝对值'] / total_abs
    sig_coef_df['归一化系数百分比'] = sig_coef_df['归一化系数绝对值'] * 100
    
    # 按系数绝对值排序
    sig_coef_df = sig_coef_df.sort_values('系数绝对值', ascending=False)
    
    # 显示结果
    print("\n归一化后的系数 (按绝对值排序):")
    print("-----------------------------------")
    significant_str = "显著 (p<0.05)" if len(sig_coef_df[sig_coef_df['p值'] < alpha]) > 0 else "无显著变量"
    print(f"总计 {len(sig_coef_df)} 个有效变量, {significant_str}")
    print("-----------------------------------")
    
    # 先显示自变量，再显示控制变量
    for var_type in ['自变量', '控制变量']:
        type_df = sig_coef_df[sig_coef_df['变量类型'] == var_type]
        if len(type_df) > 0:
            print(f"\n{var_type}:")
            for _, row in type_df.iterrows():
                sig_mark = "*" if row['p值'] < alpha else ""
                print(f"{row['变量']}: {row['系数']:.4f} (归一化: {row['归一化系数']:.4f}, {row['归一化系数百分比']:.2f}%){sig_mark}")
    
    print("-----------------------------------")
    print("* 标记表示在0.05水平上显著")
    
    # 保存到文件
    output_file = os.path.join(folder_name, "Share-归一化的模型结果.txt")
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(f"# {model_type.upper()} 模型归一化后的系数 (按绝对值排序)\n\n")
        f.write("-----------------------------------\n")
        f.write(f"总计 {len(sig_coef_df)} 个有效变量, {significant_str}\n")
        f.write("-----------------------------------\n\n")
        
        # 添加总表
        f.write("| 变量 | 变量类型 | 系数 | p值 | 归一化系数 | 归一化百分比 | 显著性 |\n")
        f.write("|------|---------|------|------|------------|------------|--------|\n")
        
        for _, row in sig_coef_df.iterrows():
            sig = "是" if row['p值'] < alpha else "否"
            f.write(f"| {row['变量']} | {row['变量类型']} | {row['系数']:.4f} | {row['p值']:.4f} | {row['归一化系数']:.4f} | {row['归一化系数百分比']:.2f}% | {sig} |\n")
        
        f.write("\n\n## 详细分析\n\n")
        
        # 先显示自变量，再显示控制变量
        for var_type in ['自变量', '控制变量']:
            type_df = sig_coef_df[sig_coef_df['变量类型'] == var_type]
            if len(type_df) > 0:
                f.write(f"### {var_type}\n\n")
                
                for _, row in type_df.iterrows():
                    sig_symbol = "*" if row['p值'] < alpha else ""
                    direction = "正向" if row['系数'] > 0 else "负向"
                    f.write(f"#### {row['变量']}{sig_symbol}\n\n")
                    f.write(f"- 系数: {row['系数']:.4f}\n")
                    f.write(f"- p值: {row['p值']:.4f}\n")
                    f.write(f"- 影响方向: {direction}\n")
                    f.write(f"- 归一化系数: {row['归一化系数']:.4f}\n")
                    f.write(f"- 归一化百分比: {row['归一化系数百分比']:.2f}%\n")
                    f.write(f"- 是否显著: {'是' if row['p值'] < alpha else '否'}\n\n")
        
        f.write("-----------------------------------\n")
        f.write("* 标记表示在0.05水平上显著\n")
    
    print(f"\n结果已保存到 {output_file}")
    
    return sig_coef_df
# 调用归一化函数
print("\n开始归一化系数分析...")
normalized_coefs = normalize_significant_coefficients(
    results=results,
    iv_list=iv_list,
    control_vars=control_vars,
    model_type=model_type,
    alpha=0.05
)



开始归一化系数分析...
注意: 没有显著的变量 (p < 0.05)，将使用所有非NaN系数变量

归一化后的系数 (按绝对值排序):
-----------------------------------
总计 20 个有效变量, 无显著变量
-----------------------------------

自变量:
季节适应性表达: -0.0766 (归一化: -0.4111, 41.11%)
设计美感表达: -0.0312 (归一化: -0.1673, 16.73%)
空间效率表达: -0.0212 (归一化: -0.1137, 11.37%)
安全性需求表达: -0.0127 (归一化: -0.0680, 6.80%)
颜色选择表达: 0.0079 (归一化: 0.0426, 4.26%)
组合灵活性表达: 0.0062 (归一化: 0.0335, 3.35%)
多功能性表达: -0.0054 (归一化: -0.0290, 2.90%)
组装便捷性表达: 0.0039 (归一化: 0.0209, 2.09%)
便携性表达: 0.0039 (归一化: 0.0207, 2.07%)
环保材质偏好: -0.0037 (归一化: -0.0198, 1.98%)
品牌声誉表达: -0.0032 (归一化: -0.0174, 1.74%)
创新功能表达: 0.0027 (归一化: 0.0144, 1.44%)
承重能力表达: -0.0024 (归一化: -0.0126, 1.26%)
耐用性表达: 0.0019 (归一化: 0.0103, 1.03%)
价格表达: 0.0015 (归一化: 0.0080, 0.80%)
层高可调节性表达: -0.0008 (归一化: -0.0045, 0.45%)
儿童友好设计表达: 0.0008 (归一化: 0.0043, 0.43%)
防尘功能表达: -0.0003 (归一化: -0.0017, 0.17%)

控制变量:
author_friends_cnt: 0.0000 (归一化: 0.0000, 0.00%)
author_followers_cnt: 0.0000 (归一化: 0.0000, 0.00%)
-----------------------------------
* 标记表示在0.05水平上显著
